# Imports

In [1]:
import numpy as np
import pandas as pd
from pcse.db import NASAPowerWeatherDataProvider
from pcse.util import reference_ET
import datetime
from matplotlib import pyplot as plt
from tqdm import tqdm as tqdm
# import yaml
import io
#from os import listdir
#from os.path import isfile, join
#import rasterio
#import rioxarray
import xarray as xr

sys.path.append("../src/")
from sarra_py import *

# Parameters

In [2]:
date_start = datetime.date(2022,1,1)
duration = 180

# path of gridded rainfall datasets
TAMSAT_path = "/mnt/d/Mes Donnees/SARRA_data-download/data/3_output/TAMSAT_v3.1_burkina_rfe_filled"
# CHIRPS_path = "/mnt/d/Mes Donnees/SARRA_data-download/data/3_output/CHIRPS_v2.0_Africa_burkina"

# path for gridded weather datasets
AgERA5_data_path = "/mnt/d/Mes Donnees/SARRA_data-download/data/3_output/AgERA5_burkina/"

# parameter file names
file_paramVariete = "USA_iowa_V42.yaml"
file_paramITK = "USA_iowa_V42.yaml"
file_paramTypeSol = "USA_iowa_V42.yaml"

# comparison with existing results : data exported from SARRA-H
file_df_weather = "Meteorologie_US0001.txt"
file_df_rain = "Pluviometrie_US0001.txt"
file_df_ET0 = "ET0_US0001.txt"
file_df_irrig = "Irrigation_AG3US20124.txt"

# Loading data

In [3]:
grid_width, grid_height = get_grid_size(TAMSAT_path, date_start, duration)

data_2 = xr.Dataset()
data_2 = load_TAMSAT_data(data_2, TAMSAT_path, date_start, duration)
data_2 = load_AgERA5_data(data_2, AgERA5_data_path, date_start, duration)
paramVariete, paramITK, paramTypeSol = load_YAML_parameters(file_paramVariete, file_paramITK, file_paramTypeSol)
data_2 = initialize_default_irrigation(data_2)
data_2 = load_iSDA_soil_data(data_2, grid_width, grid_height)

In [ ]:
data_2

<xarray.Dataset>
Dimensions:        (x: 239, y: 186, time: 180)
Coordinates:
  * x              (x) float64 -5.963 -5.925 -5.888 -5.85 ... 2.887 2.925 2.962
  * y              (y) float64 15.97 15.94 15.9 15.86 ... 9.15 9.113 9.075 9.038
    spatial_ref    int64 0
Dimensions without coordinates: time
Data variables: (12/13)
    rain           (time, y, x) float32 0.0 0.0 1.6 1.7 1.5 ... 8.6 8.1 6.8 6.2
    tpMoy          (time, y, x) float32 nan nan nan nan ... 24.54 24.54 24.23
    ET0            (time, y, x) float32 nan nan nan nan ... 2.47 2.47 2.47 2.35
    rg             (time, y, x) float32 nan nan nan nan ... 20.4 20.4 20.4 19.83
    irrigation     (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    soil_type      (y, x) float64 11.0 11.0 9.0 9.0 11.0 ... 9.0 9.0 9.0 9.0 9.0
    ...             ...
    epaisseurSurf  (y, x) float64 200.0 200.0 200.0 200.0 ... 200.0 200.0 200.0
    stockIniProf   (y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    stockIniSurf   (y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    seuilRuiss     (y, x) float64 20.0 20.0 18.0 18.0 ... 18.0 18.0 18.0 18.0
    pourcRuiss     (y, x) float64 27.0 27.0 32.0 32.0 ... 32.0 32.0 32.0 32.0
    ru             (y, x) float64 130.0 130.0 155.0 155.0 ... 155.0 155.0 155.0

# Functions

## initPlotMc

In [ ]:
def InitPlotMc(data, grid_width, grid_height, paramITK, paramTypeSol, duration):
    """
    Initializes variables related to crop residues boimass (mulch) in the data xarray dataset.
    This code has been adapted from the original InitPlotMc procedure, Bileau.pas code.
    Comments with tab indentation are from the original code.
    As the rain is the first variable to be initialized in the data xarray dataset, its dimensions are used
    to initialize the other variables.
    """

    # Initial biomass of crop residues (mulch) (kg/ha)
    # Biomasse initiale des résidus de culture (mulch) (kg/ha)
    #   BiomMc := BiomIniMc;
    data["biomMc"] = (data["rain"].dims, np.full((duration, grid_width, grid_height), paramITK["biomIniMc"]))
    data["biomMc"].attrs = {"units": "kg/ha", "long_name": "Initial biomass of crop residues (mulch)"}


    # Initial biomass of stem residues as litter (kg/ha)
    # Biomasse initiale des résidus de tiges sous forme de litière (kg/ha)
    #   LitTiges := BiomIniMc;
    data["LitTige"] = (data["rain"].dims, np.full((duration, grid_width, grid_height), paramITK["biomIniMc"]))
    data["LitTige"].attrs = {"units": "kg/ha", "long_name": "Initial biomass of stem residues as litter"}


    # ?
    #   StSurf := StockIniSurf;
    # data["stSurf"] = np.full((grid_width, grid_height, duration), paramTypeSol["stockIniSurf"])


    # ?
    #   Ltr := 1;
    data["ltr"] = (data["rain"].dims, np.full((duration, grid_width, grid_height), 1.0))


    # Soil maximum water storage capacity (mm)
    # Capacité maximale de la RU (mm)
    #   StRurMax := Ru * ProfRacIni / 1000;
    #! renaming stRurMax with root_tank_capacity
    #// data["stRurMax"] = data["ru"] * paramITK["profRacIni"] / 1000
    data["root_tank_capacity"] = data["ru"] * paramITK["profRacIni"] / 1000
    #// data["stRurMax"].attrs = {"units": "mm", "long_name": "Soil maximum water storage capacity"}
    data["root_tank_capacity"].attrs = {"units": "mm", "long_name": "Soil maximum water storage capacity"}


    # Maximum water capacity of surface tank (mm)
    # Reserve utile de l'horizon de surface (mm)
    #   RuSurf := EpaisseurSurf / 1000 * Ru;
    #! renaming ruSurf with surface_tank_capacity
    #// data["ruSurf"] = data["epaisseurSurf"] / 1000 * data["ru"]
    data["surface_tank_capacity"] = data["epaisseurSurf"] / 1000 * data["ru"]
    #// data["ruSurf"].attrs = {"units": "mm", "long_name": "Maximum water capacity of surface tank"}
    data["surface_tank_capacity"].attrs = {"units": "mm", "long_name": "Maximum water capacity of surface tank"}
    

    # ?
    #   //    PfTranspi := EpaisseurSurf * HumPf;
    #   //    StTot := StockIniSurf - PfTranspi/2 + StockIniProf;
    #   StTot := StockIniSurf  + StockIniProf;
    # data["stTot"] = np.full((grid_width, grid_height, duration), (paramTypeSol["stockIniSurf"] + paramTypeSol["stockIniProf"]))
    #! modifié pour faire correspondre les résultats de simulation, à remettre en place pour un calcul correct dès que possible
    # data["stTot"] = np.full((grid_width, grid_height, duration), (paramTypeSol["stockIniProf"]))
    #! renaming stTot to total_tank_stock
    #// data["stTot"] = data["stockIniProf"]
    data["total_tank_stock"] = data["stockIniProf"]
    #// data["stTot"].attrs = {"units": "mm", "long_name": "?"}
    data["total_tank_stock"].attrs = {"units": "mm", "long_name": "?"}
    

    # Soil maximal depth (mm)
    # Profondeur maximale de sol (mm)
    #   ProfRU := EpaisseurSurf + EpaisseurProf;
    data["profRu"] = data["epaisseurProf"] + data["epaisseurSurf"]
    data["profRu"].attrs = {"units": "mm", "long_name": "Soil maximal depth"}


    # Maximum water capacity to humectation front (mm)
    # Quantité d'eau maximum jusqu'au front d'humectation (mm)
    #   // modif 10/06/2015  resilience stock d'eau
    #   // Front d'humectation egal a RuSurf trop de stress initial
    #   //    Hum := max(StTot, StRurMax);
    #   Hum := max(RuSurf, StRurMax);
    #   // Hum mis a profRuSurf
    #   Hum := max(StTot, Hum);
    data["hum"] = (data["rain"].dims, np.full((duration, grid_width, grid_height),
        np.maximum(
            np.maximum(
                #! renaming ruSurf with surface_tank_capacity
                #// data["ruSurf"],
                data["surface_tank_capacity"],
                #! renaming stRurMax with root_tank_capacity
                #// data["stRurMax"],
                data["root_tank_capacity"],
            ),
            #! renaming stTot with total_tank_stock
            #// data["stTot"],
            data["total_tank_stock"],
        )
    ))
    data["hum"].attrs = {"units": "mm", "long_name": "Maximum water capacity to humectation front"}


    # Previous value for Maximum water capacity to humectation front (mm)
    #  HumPrec := Hum;
    data["humPrec"] = data["hum"]
    
    
    # ?
    #   StRurPrec := 0;


    # Previous value for stTot
    #   StRurMaxPrec := 0;
    #   //modif 10/06/2015 resilience stock d'eau
    #! renaming stTot with total_tank_stock
    #// data["stRuPrec"] =  data["stTot"]
    data["stRuPrec"] =  data["total_tank_stock"]
    


    return data

## InitSup

In [ ]:
def InitSup(data, grid_width, grid_height, duration, paramTypeSol, paramITK):
    """
    Initializes supplementary variables needed for computations.
    As the rain is the first variable to be initialized in the data xarray dataset, its dimensions are used
    to initialize the other variables.
    """   

    data = data.copy(deep=True)

    variables = {
        "assim": ["",""],
        "assimPot": ["",""],
        "biomTotStadeFloraison": ["",""],
        "biomTotStadeIp": ["",""],
        "bM": ["",""],
        "changePhase" : ["indicator of phase transition","binary"],
        "cM": ["",""],
        "consoRur": ["",""],
        "conv": ["",""],
        "correctedIrrigation" : ["corrected irrigation","mm"],
        "cstr" : ["drought stress coefficient", "arbitrary unit"],
        "dayBiomLeaf": ["",""],
        "dayVrac" : ["modulated daily root growth","mm/day"],
        "ddj": ["daily thermal time","°C"],
        "deltaBiomasseAerienne": ["",""],
        "deltaBiomasseFeuilles": ["",""],
        "deltaRur": ["change in root system water reserve","mm"],
        "dr": ["",""],
        "dRdtPot": ["",""],
        "dureeDuJour": ["",""],
        "eauCaptee" : ["water captured by the mulch in one day","mm"],
        "eauDispo" : ["available water, sum of rainfall and total irrigation for the day","mm"],
        "eauTranspi": ["",""],
        "etm": ["",""],
        "etp": ["",""],
        "etr": ["",""],
        "evap": ["",""],
        "evapPot": ["",""],
        "FEMcW": ["",""],
        "fesw": ["",""],
        "FeuilleUp": ["",""],
        "ftsw": ["",""],
        "initPhase": ["",""],
        "irrigTotDay" : ["total irrigation for the day","mm"],
        "KAssim": ["",""],
        "kce": ["",""],
        "kcp": ["",""],
        "kcTot": ["",""],
        "kRespMaint": ["",""],
        "LitFeuille": ["",""],
        "lr" : ["daily water runoff","mm"],
        "manqueAssim": ["",""],
        "nbJourCompte": ["",""],
        "nbjStress": ["",""],
        "NbUBT": ["",""],
        "pFact": ["",""],
        "phaseDevVeg": ["",""],
        "phasePhotoper": ["photoperiodic phase indicator","binary"],
        "rapDensite": ["",""],
        "rdt": ["",""],
        "rdtPot": ["",""],
        "reallocation": ["",""],
        "respMaint": ["",""],
        "ruIrr" : ["?","mm"],
        "ruRac": ["Water column that can potentially be strored in soil volume explored by root system","mm"],
        "seuilTempPhasePrec": ["",""],
        "sla": ["",""],
        "sommeDegresJourPhasePrec": ["",""],
        "startLock": ["",""],
        "stockIrr" : ["?","mm"],
        "stockMc" : ["water stored in crop residues (mulch)","mm"],
        "stockRac": ["",""],
        # renaming stRu to root_tank_stock
        #// "stRu": ["",""],
        "root_tank_stock": ["",""],
        "stRuMax": ["",""],
        "stRur": ["",""],
        "stRurMaxPrec": ["",""],
        "stRurPrec": ["",""],
        "stRurSurf": ["",""],
        "stRuSurf": ["",""],
        "stRuSurfPrec": ["",""],
        "stRuVar": ["",""],
        "sumPP": ["",""],
        "TigeUp": ["",""],
        "tr": ["",""],
        "trPot": ["",""],
        "trSurf": ["",""],
        "UBTCulture": ["",""],
        "vRac" : ["reference daily root growth","mm/day"],
    }
    


    for variable in variables :
        data[variable] = (data["rain"].dims, np.zeros(shape=(duration, grid_width, grid_height)))
        data[variable].attrs = {"units":variables[variable][1], "long_name":variables[variable][0]}


    data["irrigAuto"] = (data["rain"].dims, np.full((duration, grid_width, grid_height), paramITK["irrigAuto"]))
    data["irrigAuto"].attrs = {"units":"binary", "long_name":"automatic irrigation indicator"}


    return data

## evalIrrigPhase

In [ ]:
def EvalIrrigPhase(j, data, paramITK):
    """
    Translated from the procedure EvalIrrigPhase, of the original Pascal codes
    bileau.pas and exmodules2.pas.

    In irrigAuto mode, this function computes the size and filling of the
    irrigation tank, and the irrigation demand, according to the irrigation
    target (irrigAutoTarget), the maximum irrigation capacity (maxIrrig), and
    the size and filling of the root zone (stRurMax, stRur) and the surface
    reservoir (stRuSurf, ruSurf).

    It first calculates stockIrr, the water stock in the irrigation tank, and
    ruIrr, the maximum water capacity of irrigation tank. Both stockIrr and
    ruIrr are given minimum boundaries related to properties of the surface
    reservoir. Then, it calculates the irrigation demand, irrigTotDay.

    Notes from CB, 2014 :
    Modification due à la prise en compte effet Mulch Soit on a une irrigation
    observée, soit on calcul la dose d'irrigation Elle est calculée en fonction
    d'un seuil d'humidité (IrrigAutoTarget) et de possibilité technique ou choix
    (MaxIrrig, Precision) Dans cette gestion d'irrigation la pluie du jour n'est
    pas prise en compte

    N.B.: here, precision is not taken into account anymore
    """

    # managing mutability of data
    data = data.copy(deep=True)

    # First, we store initial irrigation value of the day in the
    # correctedIrrigation array
    # ! it does not seem definition and use of correctedIrrigation is useful
    # ! instead we will just use the already defined irrigation array
    # // data["correctedIrrigation"][j, :, :] = data["irrigation"][j, :, :].copy(deep=True)
    

    # Updating stockIrr : 
    #
    # stockIrr : "water stock in the irrigation tank" (mm)
    #
    # if we are in automatic irrigation mode, and between phases 0 and 6, and if
    # stRurMax (maximum water storage capacity of the root compartment, defined
    # by the root depth (mm) x the soil water storage capacity (mm/mm)) is less
    # than ruSurf (maximum water storage capacity of the surface reservoir,
    # defined by the depth of the surface horizon (mm) x the soil water storage
    # capacity (mm/mm)), meaning in the simulation that roots haven't reached
    # yet the limit between the surface compartment and deep compartment,
    # 
    # then we define stockIrr as equal to stRuSurf (water stock in the surface
    # reservoir), that is to say, we give to stockIrr a minimum value that
    # equals stRuSurf.
    # 
    # Else, we do not modify stockIrr value.
    # 
    # N.B. : for phase 7, we keep the existing stockIrr.

    condition = (data["irrigAuto"][j, :, :] == True) & \
        (data["numPhase"][j, :, :] > 0) & \
        (data["numPhase"][j, :, :] < 6)

    # group 1
    data["stockIrr"][j, :, :] = np.where(
        condition,
        np.where(
            #! renaming stRurMax to root_tank_capacity
            #! renaming ruSurf with surface_tank_capacity
            #// (data["stRurMax"] < data["ruSurf"]),
            (data["root_tank_capacity"] < data["surface_tank_capacity"]),
            #! renaming stRuSurf to surface_tank_stock
            #// data["stRuSurf"][j, :, :],
            data["surface_tank_stock"][j, :, :],
            #! renaming stRur to root_tank_stock
            #// data["stRur"][j, :, :],
            data["root_tank_stock"][j, :, :],
        ),
        data["stockIrr"][j, :, :],
    )

    # updating ruIrr :
    # 
    # ruIrr : "maximum water capacity of irrigation tank" (mm)
    #
    # if we are in automatic irrigation mode, and between phases 0 and 6, and if
    # stRurMax (water storage capacity of the root compartment, defined by the
    # root depth (mm) x the soil water storage capacity (mm/mm)) is less than
    # ruSurf (water storage capacity of the surface reservoir, defined by the
    # depth of the surface horizon (mm) x the soil water storage capacity
    # (mm/mm)), meaning that roots haven't reached the limit between the surface
    # compartment and deep compartment,
    # 
    # then we define ruIrr as equal to ruSurf (maximum water capacity of surface
    # tank (mm)), that is to say, we give to ruIrr a minimum value that equals
    # ruSurf.
    # 
    # else, we do not modify ruIrr value

    # group 2
    data["ruIrr"][j, :, :] = np.where(
        condition,
        np.where(
            #! renaming stRurMax to root_tank_capacity
            #! renaming ruSurf with surface_tank_capacity
            #// (data["stRurMax"] < data["ruSurf"]),
            (data["root_tank_capacity"] < data["surface_tank_capacity"]),
            #// data["ruSurf"],
            data["surface_tank_capacity"],
            #// data["stRurMax"],
            data["root_tank_capacity"],
        ),
        data["ruIrr"][j, :, :],
    )

    # updating irrigTotDay :
    #
    # irrigTotDay : "total irrigation need of the day"
    #
    # if we are in automatic irrigation mode, and between phases 0 and 6, and if
    # the filling of the irrigation tank is below the target filling value
    # (irrigAutoTarget, decimal percentage), then we add 90% of the difference
    # between stockIrr and ruIrr (that is to say, 90% of the volume needed to
    # fill the irrigation tank), bounded by a minimum of 0 and a maximum of
    # maxIrrig.
    # 
    # // This value is corrected correctedIrrigation.
    # This value is corrected by the already existing irrigation value.

    condition = (data["irrigAuto"][j, :, :] == True) & \
        (data["numPhase"][j, :, :] > 0) & \
        (data["numPhase"][j, :, :] < 6) & \
        (data["stockIrr"][j, :, :]/data["ruIrr"]
         [j, :, :] < paramITK["irrigAutoTarget"])

    # group 3
    data["irrigTotDay"][j, :, :] = np.where(
        condition,
        np.minimum(
            np.maximum(
                0,
                # ! replacing correctedIrrigation by irrigation
                # // ((data["ruIrr"][j, :, :] - data["stockIrr"][j, :, :]) * 0.9) - data["correctedIrrigation"][j, :, :]),
                ((data["ruIrr"][j, :, :] - data["stockIrr"][j, :, :]) * 0.9) - data["irrigation"][j, :, :]),
            paramITK["maxIrrig"]
        ),
        data["irrigTotDay"][j, :, :],
    )

    # finally, we calculate the total irrigation of the day by summing the estimated irrigation need with
    # the irrigation 
    
    # group 4
    data["irrigTotDay"][j, :, :] = (
        # ! replacing correctedIrrigation by irrigation
        # // data["correctedIrrigation"][j, :, :] + data["irrigTotDay"][j, :, :]).copy()
        data["correctedIrrigation"][j, :, :] + data["irrigTotDay"][j, :, :]).copy()

    return data


## pluieIrrig

In [ ]:
def PluieIrrig(j, data):
    """
    Translated from the procedure PluieIrrig, of the original Pascal codes
    bileau.pas and exmodules2.pas

    This function computes the total water available for the day, by summing the
    rain and the irrigation.

    Notes from CB, 2014 :
    Hypotheses : Le mulch ajoute une couche direct sous la pluie et irrig, ici
    irrigTotDay qui est l'irrigation observée ou calculée, d'où on regroupe les
    deux avant calcul de remplissage du mulch et ensuite calcul du ruissellement.

    group 5
    """

    data = data.copy(deep=True)

    data["eauDispo"][j,:,:] = data["rain"][j,:,:] + data["irrigTotDay"][j,:,:]

    return data

## rempliMc

In [ ]:
def RempliMc(j, data, paramITK):

    """
    Translated from the procedure PluieIrrig, of the original Pascal codes
    bileau.pas and exmodules2.pas

    For more details, it is advised to refer to the works of Eric Scopel (UR
    AIDA), and the PhD dissertation of Fernando Maceina. 

    Notes from CB, 2014 :

    Hypotheses :
    A chaque pluie, on estime la quantité d'eau pour saturer le couvert. On la
    retire à l'eauDispo (pluie + irrig). On calcule la capacité maximum de
    stockage fonction de la biomasse et du taux de saturation rapportée en mm
    (humSatMc en kg H2O/kg de biomasse).
    La pluie est en mm :
    1 mm = 1 litre d'eau / m2
    1 mm = 10 tonnes d'eau / hectare = 10 000 kg/ha
    La biomasse est en kg/ha pour se rapporter à la quantité de pluie captée en
    mm Kg H2O/kg Kg/ha et kg/m2 on divise par 10 000 (pour 3000 kg/ha à humSat
    2.8 kg H2O/kg on a un stockage max de 0.84 mm de pluie !?) Cette capacité à
    capter est fonction du taux de couverture du sol calculé comme le LTR SurfMc
    est spécifié en ha/t (0.39), on rapporte en ha/kg en divisant par 1000 On
    retire alors les mm d'eau captées à la pluie incidente. Le ruisselement est
    ensuite calculé avec l'effet de contrainte du mulch

    group 10
    """

    # managing mutability of data
    data = data.copy(deep=True)

    # Determination of water gathered by the mulch (eauCaptee, mm):
    #
    # We determine the quantity of water gathered by mulch by multiplying the
    # available water (eauDispo, from rain and irrigation, mm) with a
    # exponential function of covering capacity of the considered mulch (surfMc,
    # ha/t) and the mulch biomass (biomMc, kg/ha), representing the fraction of
    # soil covered by mulch. The value of eauCaptee is bounded by the maximum
    # capacity of the mulch to gather water (humSatMc, kg H2O/kg biomass), minus
    # stock of water already present in it (stockMc, mm).

    # group 7
    #! modyfing variable names to improve readability
    #! replacing eauCaptee by water_gathered_by_mulch
    #! replacing stockMc by mulch_water_stock
    #// data["eauCaptee"][j,:,:] = np.minimum(
    data["water_gathered_by_mulch"][j,:,:] = np.minimum(
        data["eauDispo"][j,:,:] * (1 - np.exp(-paramITK["surfMc"] / 1000 * data["biomMc"][j,:,:])),
        #// (paramITK["humSatMc"] * data["biomMc"][j,:,:] / 10000) - data["stockMc"][j,:,:],
        (paramITK["humSatMc"] * data["biomMc"][j,:,:] / 10000) - data["mulch_water_stock"][j,:,:],
    )

    
    # Updating available water (eauDispo, mm) : 
    #
    # As some water is gathered by the mulch, the available water is updated by
    # subtracting the gathered water (eauCaptee, mm) from the total available
    # water (eauDispo, mm). This value is bounded by 0, as the available water
    # cannot be negative.

    # ! correction as broadcasting on xarray seems less constrained than on numpy
    #! modyfing variable names to improve readability
    #! replacing eauCaptee by water_gathered_by_mulch
    # group 8
    #// data["eauDispo"][j:,:,:] =  np.maximum(data["eauDispo"][j,:,:] - data["eauCaptee"][j,:,:], 0) # //[...,np.newaxis]
    data["eauDispo"][j:,:,:] =  np.maximum(data["eauDispo"][j,:,:] - data["water_gathered_by_mulch"][j,:,:], 0) # //[...,np.newaxis]
    
    # Updating water stock in mulch (stockMc, mm) :
    #
    # The water stock in mulch is updated by adding the gathered water (eauCaptee, mm)
    
    # ! correction as broadcasting on xarray seems less constrained than on numpy
    # group 9
    #! replacing eauCaptee by water_gathered_by_mulch
    #! replacing stockMc by mulch_water_stock
    #// data["stockMc"][j:,:,:] = (data["stockMc"][j,:,:] + data["eauCaptee"][j,:,:]) # //[...,np.newaxis]
    data["stockMc"][j:,:,:] = (data["mulch_water_stock"][j,:,:] + data["water_gathered_by_mulch"][j,:,:]) # //[...,np.newaxis]

    return data

## evalRunOff   

In [ ]:
def EvalRunOff(j, data, paramTypeSol):

    """
    Translated from the procedure PluieIrrig, of the original Pascal codes
    bileau.pas, exmodules1.pas and exmodules2.pas

    Notes from CB, 2014 :
    On a regroupé avant la pluie et l'irrigation (a cause de l'effet Mulch)
    si mulch on a enlevé l'eau captée
    oN CALCUL SIMPLEMENT LE RUISSELLEMENT EN FN DE SEUILS
    }

    group 13
    """

    # managing mutability of data
    data = data.copy(deep=True)

    # evaluation of runoff ("lame de ruissellement", lr, mm) :
    # 
    # If the quantity of rain (mm) is above the runoff threshold (seuilRuiss,
    # mm), runoff is computed as the difference between the available water
    # (eauDispo, mm) and the runoff threshold (seuilRuiss, mm) multiplied by the
    # runoff percentage (pourcRuiss, %). Else, runoff value is set to 0.
    # ? Should runoff be computed taking in consideration water captured by mulch to account for mulch effect on runoff mitigation ?

    # group 11
    data["lr"][j,:,:] = np.where(
        data["rain"][j,:,:] > data["seuilRuiss"][j,:,:],
        (data["eauDispo"][j,:,:]  - data["seuilRuiss"][j,:,:]) * data["pourcRuiss"][j,:,:],
        # // data["lr"][j,:,:],
        0,
    )

    # Updating available water (eauDispo, mm) :
    #
    # The available water is updated by subtracting the runoff (lr, mm) from the
    # total available water (eauDispo, mm). This value is broadcasted onto the
    # days axis.

    # group 12
    data["eauDispo"][j:,:,:] = (data["eauDispo"][j,:,:] - data["lr"][j,:,:]).copy()[...,np.newaxis]

    return data


## evolRurCstr2

In [ ]:
def EvolRurCstr2(j, data, paramITK):

    """
    Translated from the procedure PluieIrrig, of the original Pascal codes
    bileau.pas

    Notes from CB, 10/06/2015 :
    Stress trop fort enracinement
    Trop d'effet de stress en tout début de croissance :
    1) la plantule a des réserves et favorise l'enracinement
    2) dynamique spécifique sur le réservoir de surface
    Cet effet stress sur l'enracinement ne s'applique que quand l'enracinement
    est supérieur é la profondeur du réservoir de surface. Effet stres a un
    effet sur la vitesse de prof d'enracinement au dessus d'un certain seuil de
    cstr (on augmente le cstr de 0.3 pour que sa contrainte soit affaiblie sur
    la vitesse) La vitesse d'enracinement potentielle de la plante peut etre
    bloque par manque d'eau en profondeur (Hum). La profondeur d'humectation est
    convertie en quantite d'eau maximum equivalente

    IN:
    Vrac : mm (en mm/jour) : Vitesse racinaire journalière §§ Daily root depth
    Hum : mm Quantité d'eau maximum jusqu'au front d'humectation §§ Maximum
    water capacity to humectation front
    StRuSurf : mm
    RU : mm/m
    RuSurf : mm/m

    INOUT:
    stRurMax : mm ==== ruRac
    stRur : mm ==== stockRac

    NB : on remet le nom de variables de CB plutôt que celles utilisées par MC dans le code Java
    """

    # managing mutability of data
    data = data.copy(deep=True)

    # ! dayvrac et deltarur reset à chaque itération ; on traine donc le j sur les autres variables

    # updating stRurMax, step 1 :
    #
    # stRurMax, also called ruRac in some versions of the model, is the maximum
    # root water storage capacity, aka Water column that can potentially be
    # strored in soil volume explored by root system.
    #
    # At the phase change between phases 0 and 1 (initialisation), the maximum
    # root water storage is initialised by multiplying the initial root depth
    # (profRacIni, mm) with the soil water storage capacity (ru, mm/m). This
    # value is broadcasted on the time series. For every other day in the cycle,
    # the value remains unchanged.

    #! renaming stRurMax to root_tank_capacity
    # group 14
    #// data["stRurMax"][j:,:,:] = np.where(
    data["root_tank_capacity"][j:,:,:] = np.where(
        (data["changePhase"][j,:,:] == 1) & (data["numPhase"][j,:,:] == 1),
        paramITK["profRacIni"] / 1000 * data["ru"][j,:,:],
        #// data["stRurMax"][j,:,:],
        data["root_tank_capacity"][j,:,:],
    )[...,np.newaxis]


    # updating dayVrac (daily variation in water height accessed by roots,
    # mm/day) :
    # 
    # At the day of phase change, for phases strictly above 1, and for which
    # maximum root water storage capacity (stRurMax) is greater than ruSurf
    # (maximum water storage capacity of the surface reservoir), the daily
    # variation in water height accessed by roots (dayVrac) is computed as the
    # product of the root growth speed (vRac, mm/day) and a coefficient. The
    # coefficient is computed as the drought stress coefficient (cstr) plus 0.3,
    # with a maximum bound of 1.0. It is thus modulated by drought stress. That
    # is to say, when the root has a maximum root water storage capacity above
    # the maximum water storage capacity of the surface reservoir, the root
    # growth speed is modulated by drought stress.
    #
    # ? Why is dayVrac in this case bounded in [0.3, 1] ?
    # According to CB, this is based on the hypothesis that during a drought
    # stress, the plant will preferably grow roots. Furthermore, using the [0.3,
    # 1] bound is a way to tell that up to a cstr of 0.7 there is no effect of
    # drought stress on the root growth speed.
    #
    # At the day of phase change, for phases strictly above 1, and for which
    # stRurMax is NOT greater than ruSurf, dayVrac is computed as the product of
    # the root growth speed (vRac, mm/day) and ru (mm/m).
    # 
    # Every other day, dayVrac is unchanged.

    # group 15  
    # ! simplified conditions
    # // condition = (data["numPhase"][j,:,:] > 0) & \
    # //       np.invert((data["numPhase"][j,:,:] == 1) & (data["changePhase"][j,:,:] == 1))
    condition = (data["numPhase"][j,:,:] > 1) & (data["changePhase"][j,:,:] == 1)

    #! renaming dayVrac to delta_root_tank_capacity
    #// data["dayVrac"][j,:,:] = np.where(
    data["delta_root_tank_capacity"][j,:,:] = np.where(
        condition,
        np.where(
            #! renaming stRurMax to root_tank_capacity
            #! renaming ruSurf to surface_tank_capacity
            #// (data["stRurMax"][j,:,:] > data["ruSurf"][j,:,:]),
            (data["root_tank_capacity"][j,:,:] > data["surface_tank_capacity"][j,:,:]),
            (data["vRac"][j,:,:] * np.minimum(data["cstr"][j,:,:] + 0.3, 1.0)) / 1000 * data["ru"][j,:,:],
            data["vRac"][j,:,:] / 1000 * data["ru"][j,:,:],
        ),
        #// data["dayVrac"][j,:,:],
        data["delta_root_tank_capacity"][j,:,:],
    )

    
    # updating deltaRur (change in root water storage capacity, mm) : 
    #
    # At the day of phase change, for phases strictly above 1, and for which the
    # difference between the humectation front (hum, mm) and the maximum root
    # water storage capacity (stRurMax, mm) is less than the daily variation in
    # water height accessed by roots (dayVrac, mm/day), the change in root water
    # storage capacity (deltaRur) is computed as the difference between the
    # humectation front and the maximum root water storage capacity. That is to
    # say that change in root water storage capacity is limited by the
    # humectation front.
    # 
    # At the day of phase change, for phases strictly above 1, and for which the
    # difference between the humectation front (hum, mm) and the maximum root
    # water storage capacity (stRurMax, mm) is NOT LESS than the daily variation
    # in water height accessed by roots (dayVrac, mm/day), the change in root
    # water storage capacity (deltaRur) is computed as the daily variation in
    # water height accessed by roots (dayVrac, mm/day).
    # 
    # For any other day, the change in root water storage capacity (deltaRur) is
    # unchanged.

    # group 16
    # ! simplified conditions
    # // condition = (data["numPhase"][j,:,:] > 0) & \
    # //       np.invert((data["numPhase"][j,:,:] == 1) & (data["changePhase"][j,:,:] == 1))
    condition = (data["numPhase"][j,:,:] > 1) & (data["changePhase"][j,:,:] == 1)

    #! renaming deltaRur with delta_root_tank_capacity
    #// data["deltaRur"][j:,:,:] = np.where(
    data["delta_root_tank_capacity"][j:,:,:] = np.where(
        condition,   
        np.where(
            #! renaming stRurMax to root_tank_capacity
            #! renaming dayVrac to delta_root_tank_capacity
            #// (data["hum"][j,:,:] - data["stRurMax"][j,:,:]) < data["dayVrac"][j,:,:],
            (data["hum"][j,:,:] - data["root_tank_capacity"][j,:,:]) < data["delta_root_tank_capacity"][j,:,:],
            #// data["hum"][j,:,:] - data["stRurMax"][j,:,:],
            data["hum"][j,:,:] - data["root_tank_capacity"][j,:,:],
            #! renaming dayVrac to delta_root_tank_capacity
            #// data["dayVrac"][j,:,:],
            data["delta_root_tank_capacity"][j,:,:],
        ),
        #// data["deltaRur"][j,:,:],
        data["delta_root_tank_capacity"][j,:,:],
    )[...,np.newaxis]


    # updating stRurMax/ruRac, step 2 :
    # 
    # At the day of phase change, for phases strictly above 1, the maximum root
    # water storage capacity (stRurMax, mm) is updated to be summed with the change
    # in root water storage capacity (deltaRur, mm). That is to say that the maximum
    # root water storage capacity is incremented by the change in root water
    # storage capacity.
    # 
    # stRurMax value is broadcasted 

    # group 17
    # ! simplified conditions
    # // data["stRurMax"][j:,:,:] = np.where(
    # //     (data["numPhase"][j,:,:] > 0),
    # //     np.where(
    # //         np.invert((data["changePhase"][j,:,:] == 1) & (data["numPhase"][j,:,:] == 1)),
    # //         data["stRurMax"][j,:,:] + data["deltaRur"][j,:,:],
    # //         data["stRurMax"][j,:,:],
    # //     ),
    # //     data["stRurMax"][j,:,:],
    # // )[...,np.newaxis]
    #! renaming stRurMax to root_tank_capacity
    #! renaming deltaRur to delta_root_tank_capacity
    #// data["stRurMax"][j:,:,:] = np.where(
    data["root_tank_capacity"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] > 1) & (data["changePhase"][j,:,:] == 1),
        #// data["stRurMax"][j,:,:] + data["deltaRur"][j,:,:],
        data["root_tank_capacity"][j,:,:] + data["delta_root_tank_capacity"][j,:,:],
        #// data["stRurMax"][j,:,:],
        data["root_tank_capacity"][j,:,:],
    )[...,np.newaxis]

    
    # updating stRur/stockrac :
    #
    # At the day of phase change, for phases strictly above 1, and for which the
    # the maximum root water storage capacity (stRurMax, mm) is above the
    # surface water storage capacity (ruSurf, mm) (meaning that roots go beyond
    # the surface water storage capacity), the root water storage capacity
    # (stRur, mm) is incremented by the change in root water storage capacity
    # (deltaRur, mm).
    # 
    # However, if the maximum root water storage capacity (stRurMax, mm) is
    # BELOW the surface water storage capacity (ruSurf, mm) (meaning that roots
    # stay within the surface water storage capacity), the root water storage
    # capacity (stRur, mm) is updated to be equal to the stRuSurf (mm) minus
    # 1/10th of the surface water storage capacity (ruSurf, mm),  multiplied by
    # the ratio between the maximum root water storage capacity (stRurMax, mm)
    # and the surface water storage capacity (ruSurf, mm). That is to say "we
    # take at the prorata of depth and surface stock"
    #
    # For any other day, the root water storage capacity (stRur, mm) is
    # unchanged.
    #
    # stRur value is broadcasted

    # group 18
    # ! simplified conditions
    # // condition = (data["numPhase"][j,:,:] > 0) & np.invert((data["changePhase"][j,:,:] == 1) & (data["numPhase"][j,:,:] == 1)),
    condition = (data["numPhase"][j,:,:] > 1) & (data["changePhase"][j,:,:] == 1),
    
    #! renaming stRur to root_tank_stock
    #// data["stRur"][j:,:,:] = np.where(
    data["root_tank_stock"][j:,:,:] = np.where(
        condition,
        np.where(
            #! renaming stRurMax to root_tank_capacity
            #! renaming ruSurf to surface_tank_capacity
            #// (data["stRurMax"][j,:,:] > data["ruSurf"][j,:,:]),
            (data["root_tank_capacity"][j,:,:] > data["surface_tank_capacity"][j,:,:]),
            #! renaming stRur to root_tank_stock
            #! renaming deltaRur to delta_root_tank_capacity
            #// data["stRur"][j,:,:] + data["deltaRur"][j,:,:],
            data["root_tank_stock"][j,:,:] + data["delta_root_tank_capacity"][j,:,:],
            #! renaming stRur to root_tank_stock
            #! renaming stRuSurf to surface_tank_stock
            #// np.maximum((data["stRuSurf"][j,:,:] - data["ruSurf"][j,:,:] * 1/10) * (data["stRurMax"][j,:,:] / data["ruSurf"][j,:,:]), 0),
            np.maximum((data["surface_tank_stock"][j,:,:] - data["surface_tank_capacity"][j,:,:] * 1/10) * (data["root_tank_capacity"][j,:,:] / data["surface_tank_capacity"][j,:,:]), 0),
        ),  
        #! renaming stRur to root_tank_stock
        #// data["stRur"][j,:,:],
        data["root_tank_stock"][j,:,:],
    )[...,np.newaxis]
    

    return data

## rempliRes

In [ ]:
# bhytypeFAO, ***bileau***; exmodules 1 & 2, risocas ###trad OK

def rempliRes(j, data):

    # TODO: modify positions of i indexers to suit the new data structure
    """
    Translated from the procedure rempliRes, of the original Pascal codes
    bileau.pas

    Main hypotheses : 
    - the water dynamics is represented by a filling from the top and an evolution 
    of the reservoirs sizes when the filling is above the maximum quantity of the
    current size (humectation front).
    - when the maximum size is reached by filling, it is considered as drainage.
    - inside a reservoir, water is distributed homogeneously (may be considered
    valid up to 2m depth, according to CB, from other sources).

    3 reservoirs are represented:
    1) a global reservoir, evolving in depth according to the humectation front
    2) a surface reservoir (fixed size) where evaporation and a part of the
    transpiration occurs when roots are present
    3) a root reservoir, evolving according to the root front (when roots are
    present)

    REMARK : these reservoirs overlap, and instead of managing depths, we manage water stocks



    Notes from CB, 10/06/2015 :
    prise en compte de stock d'eau résilient pour les simulation continues
    Hypothèse de la MAJ des stock en fn de l'eau r�siliente de l'ann�e pr�c�dente
    dans le cas des simulations pluri annuelle en continue (NbAn = 1):
    A la r�colte on recup�re les stock d'eau (StRuPrec), la prof d'Humectation (Humprec)
    et la prof d'enracinement (stRurMaxprec). Pour le reservoir de surface on ne change rien.
    On MAJ le stRu avec le stock de surface stRuSurf, Hum avec le max de remplissage de surface (RuSurf)
    Si le StRu avec l'apport d'eau devinet sup au Hum
    alors on tient compte dans cette augmentation du stock r�silient avec deux cas possible :
    Si StRu est < � stRurMaxprec
    alors on ajoute l'eau r�siliente contenue dans l'ancienne zone racinaire en fn
    de la diff�rence de stock
    Sinon on a de l'eau r�siliente au maximum de la CC jusqu'� l'ancienne HumPrec,
    on rempli alors StRu de la diff�rence etre ces deux valeurs puis on fait la MAJ
    des Dr, StRur, Hum etc...
    """


    # updating variables at the end of the growing cycle

    condition = (data["numPhase"][j,:,:] == 7) & (data["changePhase"][j,:,:] == 1)

    # when the phase changes from 7 to 1, the humPrec (mm, previous maximum
    # water capacity to humectation front) is set to equal the higher value
    # between hum (mm, maximum water capacity to humectation front) and the root
    # water storage capacity (stRur, mm).
    # group 20
    data["humPrec"][j:,:,:] = np.where(
        condition,
        #! renaming ruSurf to surface_tank_capacity
        #// np.maximum(data["hum"][j,:,:], data["ruSurf"][j,:,:]),
        np.maximum(data["hum"][j,:,:], data["surface_tank_capacity"][j,:,:]),
        data["humPrec"][j,:,:],
    )[...,np.newaxis]

    
    # when the phase changes from 7 to 1, the hum (mm, maximum water capacity to
    # humectation front) is set to equal the ruSurf (mm, maximum water storage
    # capacity of the surface reservoir)
    # group 21
    data["hum"][j:,:,:] = np.where(
        condition,
        #! renaming ruSurf to surface_tank_capacity
        #// data["ruSurf"][j,:,:],
        data["surface_tank_capacity"][j,:,:],
        data["hum"][j,:,:],
    )[...,np.newaxis]


    # when the phase changes from 7 to 1, the stRurMaxPrec (mm, previous maximum
    # water capacity to root front) is set to equal stRurMax (mm, maximum water
    # capacity to root front)
    # group 22
    data["stRurMaxPrec"][j:,:,:] = np.where(
        condition,
        #! renaming stRurMax to root_tank_capacity
        #// data["stRurMax"][j,:,:],
        data["root_tank_capacity"][j,:,:],
        data["stRurMaxPrec"][j,:,:],
    )[...,np.newaxis]


    # when the phase changes from 7 to 1, the stRurPrec (mm, previous water
    # storage capacity of the root reservoir) is set to equal stRur/stRurMax,
    # that is to say the ratio of the water storage capacity of the root
    # reservoir
    # group 23
    data["stRurPrec"][j:,:,:] = np.where(
        condition,
        #! renaming stRur to root_tank_stock
        #! renaming stRurMax to root_tank_capacity
        #// data["stRur"][j,:,:]/data["stRurMax"][j,:,:],
        data["root_tank_stock"][j,:,:]/data["root_tank_capacity"][j,:,:],
        data["stRurPrec"][j,:,:],
    )[...,np.newaxis]


    # when the phase changes from 7 to 1, the stRuPrec (mm, previous water
    # storage capacity of the global reservoir) is set to equal the differe,ce
    # between stTot (mm, total water storage capacity of the global reservoir)
    # and stRurSurf (mm, water storage capacity of the surface reservoir)
    # group 24
    #! stRurSurf is not defined... we may want to drop this group
    data["stRuPrec"][j:,:,:] = np.where(
        condition,
        #! renaming stTot to total_tank_stock
        #// data["stRu"][j,:,:] - data["stRurSurf"][j,:,:],
        data["total_tank_stock"][j,:,:] - data["stRurSurf"][j,:,:], # essai stTot
        data["stRuPrec"][j,:,:],
    )[...,np.newaxis]







    # updating stRuMax :
    #
    # stRuMax is the max water storage capacity of the global reservoir ; it is
    # redfined at each loop as the product of the water storage capacity
    # (réserve utile, mm/m if soil) and the maximum depth of soil N.B. : on code
    # version 10/06/2015, resilience of water stock for multi-annual simulations
    # was been implemented
    #? where is it now ? why redfining stRuMax at each loop ? neither ru, profRu
    #? nor stRuMax seem to change during the simulation
    # group 25
    data["stRuMax"][j:,:,:] = (data["ru"][j,:,:] * data["profRu"][j,:,:] / 1000).copy()[...,np.newaxis]
    

    # updating stRuSurfPrec :
    #
    # stRuSurf is the water stock in the surface reservoir. here, we save the
    # previous value of stRuSurf before doing any update
    #? this has little sense, as previous value of stRuSurf is stored at the ?
    #? (i-1)-th position in the xarray dataarray
    # group 26 
    #! after replacing data["stRuSurfPrec"][j,:,:] by data["stRuSurf"][j,:,:-1]
    #! in group 28, we can safely remove the use of stRuSurfPrec
    #// data["stRuSurfPrec"][j,:,:] = data["stRuSurf"][j,:,:].copy()
    

    # updating stRuSurf :
    #
    # we update stRuSurf (mm, water stock in the surface reservoir) by adding
    # the eauDispo, which as this point is the water available from rain and
    # irrigation for the day after estimation of intake by mulch, and runoff.
    # however, we do not allow stRuSurf to exceed 110% of the ruSurf (mm,
    # maximum water storage capacity of the surface reservoir)
    # group 27
    #! renaming stRuSurf to surface_tank_stock
    #// data["stRuSurf"][j:,:,:] = np.minimum(
    data["surface_tank_stock"][j:,:,:] = np.minimum(
        #// data["stRuSurf"][j,:,:] + data["eauDispo"][j,:,:],
        data["surface_tank_stock"][j,:,:] + data["eauDispo"][j,:,:],
        #! renaming ruSurf to surface_tank_capacity
        #// 1.1 * data["ruSurf"][j,:,:]
        1.1 * data["surface_tank_capacity"][j,:,:]
    )[...,np.newaxis]


    # updating eauTranspi : estimation of transpirable water
    #
    # eauTranspi (mm, water transpirable) is the water available for
    # transpiration from the surface reservoir.
    # If the stock of surface reservoir is lower than 10% of the maximum water
    # storage capacity of the surface reservoir, the water available for
    # transpiration is the water available for the day (eauDispo), minus the
    # difference between 1/10th of the maximum water storage capacity of the
    # surface reservoir and the stock of surface reservoir, bounded by 0. Said
    # otherwise, a part of the water available for the day is considered as
    # linked to the surface reservoir.
    # Otherwise, the water available for transpiration is equal to the water
    # available for the day (eauDispo).
    #? here we use the stRuSurf value at indice j-1 because we want to use the
    #? values before their update in group 27 ; instead it may be clever to move
    #? this group before group 27 so we remove the j-1 indices
    # group 28
    data["eauTranspi"][j:,:,:] = np.where(
        # ! modifying to replace stRuSurfPrec by stRuSurf
        #! renaming ruSurf to surface_tank_capacity
        #! renaming stRuSurfPrec to surface_tank_stock
        # // data["stRuSurfPrec"][j,:,:] < data["ruSurf"][j,:,:]/10,
        data["surface_tank_stock"][j-1,:,:] < data["surface_tank_capacity"][j,:,:] * 0.1,
        np.maximum(
            0,
            # ! modifying to replace stRuSurfPrec by stRuSurf
            #! renaming ruSurf to surface_tank_capacity
            #! renaming stRuSurf to surface_tank_stock
            # //data["eauDispo"][j,:,:] - (data["ruSurf"][j,:,:]/10 - data["stRuSurfPrec"][j,:,:])
            data["eauDispo"][j,:,:] - (data["surface_tank_capacity"][j,:,:] * 0.1 - data["surface_tank_stock"][j-1,:,:])
            ),
        data["eauDispo"][j,:,:],
    )[...,np.newaxis]



    # updating stTot, step 1 :
    #
    # stTot, also known as stRu in some versions of the model; is the water
    # stock of the global reservoir. Here, we increment stTot by the quantity of
    # transpirable water.
    #? why incrementing stTot by eauTranspi ? eauTranspi is not intended to be
    #? transpired...?
    # group 29
    #! renaming stTot with total_tank_stock
    #// data["stTot"][j:,:,:] = (data["stTot"][j,:,:] + data["eauTranspi"][j,:,:]).copy()[...,np.newaxis]
    data["total_tank_stock"][j:,:,:] = (data["total_tank_stock"][j,:,:] + data["eauTranspi"][j,:,:]).copy()[...,np.newaxis]


    # defining stRuVar :
    #
    # stRuVar is the difference between the total water stock (stTot) and stRuPrec (mm, the prvious water stock of the global reservoir), bounded by 0.
    #? we may want to rename stRuPrec to stTotPrec, or use stTot with another notation
    #? stRuPrec is updated in group 24

    # modif 10/06/2015 Resilience stock eau cas simul pluri en continue
    # différence entre stock total et stRuPrec (non défini clairement ?), borné au minimum en 0
    # group 30
    #! we propose a different version based on stTot
    #! renaming stTot to total_tank_stock
    #// data["stRuVar"][j:,:,:] = np.maximum(0, data["stTot"][j,:,:] - data["stRuPrec"][j,:,:])[...,np.newaxis]
    data["stRuVar"][j:,:,:] = np.maximum(0, data["total_tank_stock"][j,:,:] - data["total_tank_stock"][j,:,:-1])[...,np.newaxis]






    condition_1 = (data["stRuVar"][j,:,:] > data["hum"][j,:,:])
    #! we replace stRurMaxPrec by stRurMax with indice j-1
    #! renaming stRurMax with root_tank_capacity
    #// condition_2 = (data["hum"][j,:,:] <= data["stRurMaxPrec"][j,:,:])
    condition_2 = (data["hum"][j,:,:] <= data["root_tank_capacity"][j,:,:-1])
    #! we replace humPrec by hum with indice j-1
    #// condition_3 = (data["hum"][j,:,:] < data["humPrec"][j,:,:])
    condition_3 = (data["hum"][j,:,:] < data["hum"][j,:,:-1])

    # updating stTot step 1
    #
    # stTot, also known as stRu in some versions of the model, is the water stock of the global reservoir.
    # if the variation in total water stock (stRuVar) is greater than the maximum water quantity until the humidity front (hum),
    # and if the quantity of water until the humidity front is equal or less than the maximum root water stock (stRurMax) at j-1,
    # the total water stock is incremented by the difference between stRuVar and hum, multiplied by the previous root water stock (stRurPrec).
    #
    # if the variation in total water stock (stRuVar) is greater than the maximum water quantity until the humidity front (hum),
    # and if the quantity of water until the humidity front is MORE than the maximum root water stock (stRurMax) at j-1,
    # then if the humidity front is lower than the previous humidity front, stTot taks stRuVar as value.
    # Otherwise, its value does not change 
    # group 31
    #! renaming stTot to total_tank_stock
    #// data["stTot"][j:,:,:] = np.where(
    data["total_tank_stock"][j:,:,:] = np.where(
        condition_1,
        np.where(
            condition_2,
            #! we replace stRurPrec with stRur at indice j-1
            #! renaming stRur to root_tank_stock
            #! renaming stTot to total_tank_stock
            #// data["stTot"][j,:,:] + (data["stRuVar"][j,:,:] - data["hum"][j,:,:]) * data["stRurPrec"][j,:,:],
            data["total_tank_stock"][j,:,:] + (data["stRuVar"][j,:,:] - data["hum"][j,:,:]) * data["root_tank_stock"][j,:,:-1],
            np.where(
                condition_3,
                data["stRuVar"][j,:,:],
                #! renaming stTot to total_tank_stock
                #// data["stTot"][j,:,:],
                data["total_tank_stock"][j,:,:],
            ),
        ),
        #! renaming stTot to total_tank_stock  
        #// data["stTot"][j,:,:],
        data["total_tank_stock"][j,:,:],
    )[...,np.newaxis]




    # group 32
    data["stRuPrec"][j:,:,:] = np.where(
        condition_1,
        np.where(
            condition_2,
            np.maximum(0, data["stRuPrec"][j,:,:] - (data["stRuVar"][j,:,:] - data["hum"][j,:,:]) * data["stRurPrec"][j,:,:]),
            np.where(
                condition_3,
                0,
                data["stRuPrec"][j,:,:],
            ),
        ),
        data["stRuPrec"][j,:,:],
    )[...,np.newaxis]



    # condition_1 = (data["stRuVar"][j,:,:] > data["hum"][j,:,:])
    # condition_2 = (data["hum"][j,:,:] <= data["stRurMaxPrec"][j,:,:])
    # condition_3 = (data["hum"][j,:,:] < data["humPrec"][j,:,:])


    # groupe 33
    data["stRuVar"][j:,:,:] = np.where(
        condition_1,
        np.where(
            condition_2,
            data["stRuVar"][j,:,:] + (data["stRuVar"][j,:,:] - data["hum"][j,:,:]) * data["stRurPrec"][j,:,:],
            np.where(
                condition_3,
                data["stRuVar"][j,:,:] + data["stRuPrec"][j,:,:],
                data["stRuVar"][j,:,:],
            ),
        ),
        data["stRuVar"][j,:,:],
    )[...,np.newaxis]

    # hum
    # front d'humectation mis à jour sur la base du delta maximal de stock d'eau total
    # dans l'intervalle [stRuVar, stRuMax]
    # modif 10/06/2015 Resilience stock eau cas simul pluri en continue
    # modif 27/07/2016 Hum ne peut �tre au dessus de stRu (stocktotal)
    # groupe 34
    data["hum"][j:,:,:] = np.maximum(data["stRuVar"][j,:,:], data["hum"][j,:,:])[...,np.newaxis]
    # groupe 35
    data["hum"][j:,:,:] = np.minimum(data["stRuMax"][j,:,:], data["hum"][j,:,:])[...,np.newaxis]


    #! renaming stTot to total_tank_stock
    #// condition = (data["stTot"][j,:,:] > data["stRuMax"][j,:,:])
    condition = (data["total_tank_stock"][j,:,:] > data["stRuMax"][j,:,:])

    # groupe 36
    # essais stTot
    data["dr"][j,:,:] = np.where(
        condition,
        #! renaming stTot to total_tank_stock
        #// data["stRu"][j,:,:] - data["stRuMax"][j,:,:],
        data["total_tank_stock"][j,:,:] - data["stRuMax"][j,:,:],
        0,
    )

    # groupe 37
    # essais stTot
    #! renaming stTot to total_tank_stock
    #// data["stRu"][j,:,:] = np.where(
    #// data["stTot"][j:,:,:] = np.where(   
    data["total_tank_stock"][j:,:,:] = np.where(  
        condition,
        data["stRuMax"][j,:,:],
        # data["stRu"][j,:,:],
        #// data["stTot"][j,:,:],
        data["total_tank_stock"][j,:,:],
    )[...,np.newaxis]


    # groupe 38
    # // avant modif 10/06/2015
    # data["hum"][j:,:,:] = np.maximum(data["hum"][j,:,:], data["stRu"][j,:,:])
    # essais stTot
    #! renaming stTot to total_tank_stock
    #// data["hum"][j:,:,:] = np.maximum(data["hum"][j,:,:], data["stTot"][j,:,:])[...,np.newaxis]
    data["hum"][j:,:,:] = np.maximum(data["hum"][j,:,:], data["total_tank_stock"][j,:,:])[...,np.newaxis]
    #! en conflit avec le calcul précédent de hum

    # groupe 39
    # Rempli res racines
    #! renaming stRur to root_tank_stock
    #! renaming stRurMax to root_tank_capacity
    #// data["stRur"][j:,:,:] = np.minimum(data["stRur"][j,:,:] + data["eauTranspi"][j,:,:], data["stRurMax"][j,:,:])[...,np.newaxis]
    data["root_tank_stock"][j:,:,:] = np.minimum(data["root_tank_stock"][j,:,:] + data["eauTranspi"][j,:,:], data["root_tank_capacity"][j,:,:])[...,np.newaxis]

    # groupe 40
    # essais stTot
    #! renaming stRur to root_tank_stock
    #! renaming stTot to total_tank_stock
    #// data["stRur"][j,:,:] = np.minimum(data["stRur"][j,:,:], data["stRu"][j,:,:])
    data["root_tank_stock"][j:,:,:] = np.minimum(data["root_tank_stock"][j,:,:], data["total_tank_stock"][j,:,:])[...,np.newaxis]

    

    return data

In [ ]:
def reset(j, data):

  data = data.copy(deep=True)

  # when reaching stage 7, we reset the main phenological variables to zero
  data["changePhase"][j:,:,:] = np.where(data["numPhase"][j,:,:] == 7, 0, data["changePhase"][j,:,:])[np.newaxis,...]
  data["sdj"][j:,:,:] = np.where(data["numPhase"][j,:,:] == 7, 0, data["sdj"][j,:,:])[np.newaxis,...]
  data["ruRac"][j:,:,:] = np.where(data["numPhase"][j,:,:] == 7, 0, data["numPhase"][j,:,:])[np.newaxis,...]
  data["nbJourCompte"][j:,:,:] = np.where(data["numPhase"][j,:,:] == 7, 0, data["numPhase"][j,:,:])[np.newaxis,...]
  data["startLock"][j:,:,:] = np.where(data["numPhase"][j,:,:] == 7, 1, data["startLock"][j,:,:])[np.newaxis,...]
  # and we leave numPhas last
  data["numPhase"][j:,:,:] = np.where(data["numPhase"][j,:,:] == 7, 0, data["numPhase"][j,:,:])[np.newaxis,...]

  return data



def EvalPhenoSarrahV3(j, data, paramITK, paramVariete): 
  
  """
  Translated from the EvalPhenoSarrahV3 procedure of the phenologie.pas and exmodules.pas files of the Sarra-H model, Pascal version.

  Cette procédure est appelée en début de journée et fait évoluer les phases
  phénologiques. Pour celà, elle incrémente les numéro de phase et change la
  valeur du seuil de somme de degré jours de la phase suivante.
  ChangePhase est un booléen permettant d'informer le modéle pour connaître
  si un jour est un jour de changement de phase. Cela permet d'initialiser les variables directement dans les  modules spécifiques.
  Méthode générique pour le test de fin de la phase photopériodique.
  PhasePhotoper = 0 en fin de la phase photoper et = 1 en debut de la phase

  This procedure is called at the beginning of the day to evolve the phenological phases.
  To do so, it computes if the sum of thermal time reaches the threshold needed for the next phenological phase.
  ChangePhase is a boolean informing the model to know if a day is a day of phase change.
  If yes, it increments the phase number and updates thermal time threshold needed to reach the next phase.
  
  PhasePhotoper = 0 at the end of the photoper phase and = 1 at the beginning of the phase

  Phenological phases used in this model (as for cereal crops) :

  0 : from the sowing day to the beginning of the conditions favorable for germination,
      and from the harvest to the end of the simulation (no crop)
  1 : from the beginning of the conditions favorable for germination to the day of germination
      (du début des conditions favorables pour la germination au jour de la levée)
  2 : from the day of germination to the beginning of the photoperiodic phase
      (du jour de la levée au début de la phase photopériodique)
  3 : from the beginning of the photoperiodic phase to the beginning of the reproductive phase
  4 : from the beginning of the reproductive phase to the beginning of the maturation (only for maize and rice) 
  5 : from the beginning of the maturation to the grain milk stage
      (du début de la maturation au stade grain laiteux)
  6 : from the grain milk stage to the end of the maturation
      (du début du stade grain laiteux au jour de récolte)
  7 : the day of the harvest
  
  In the case of multiannual continuous simulations, we do not reinitialize the reservoirs, at harvest we put the moisture front at the depth of the surface reservoir
  This allows to keep the rooting constraint phenomenon for the following season if there is little rain
  while having the water stock in depth remaining from the previous season.
  """

  # performing deepcopy to manage mutability of the xarray dataset
  data = data.copy(deep=True)

  data = reset(j, data)

  


  ### initialisation phase
  # we test if we are in the phase 0 and the water content is sufficient
  #! replacing stRuSurf by root_tank_stock
  #! renaming ruSurf with surface_tank_capacity
  condition = \
    (data["numPhase"][j,:,:] == 0) & \
    (data["root_tank_stock"][j,:,:] - data["surface_tank_capacity"] / 10 >= paramITK["seuilEauSemis"]) & \
    (data["startLock"][j,:,:] == 0)
    # ? the following code was used previously but it seems to be wrong
    # (data["stSurf"][j,:,:] - data["ruSurf"][j,:,:] / 10 >= paramITK["seuilEauSemis"])
  
  # if conditions are met,
  # we force the phase number to 1,
  # we flag a change of phase, that allows to trigger the update of the thermal time sum of the next phase
  data["numPhase"][j:,:,:] = np.where(condition, 1, data["numPhase"][j,:,:])[np.newaxis,...]
  data["changePhase"][j,:,:] = np.where(condition, 1, data["changePhase"][j,:,:])

  # we force the sum of thermal time of the next phase to be "SDJ levée"
  data["seuilTempPhaseSuivante"][j:,:,:] = np.where(
    condition,
    paramVariete["SDJLevee"],
    data["seuilTempPhaseSuivante"][j,:,:],
  )[np.newaxis,...]

  # we flag this phase change as related to initiation
  # that will contribute later on to the bypassing of phase number incrementation
  # ? keeping the following line as reference
  # data["initPhase"] = data["numPhase"].copy() * 0
  data["initPhase"][j:,:,:] = np.where(
    condition,
    1,
    data["initPhase"][j:,:,:]
  )#[np.newaxis,...]
  
 

  ### Phase transitions
  # Testing for phase 2
  condition = \
    (data["numPhase"][j,:,:] == 2) & \
    (data["sdj"][j,:,:] >= data["seuilTempPhaseSuivante"][j,:,:])

  data["changePhase"][j,:,:] = np.where(condition, 1, data["changePhase"][j,:,:])

  ### Test phases 1, 4, 5, 6
  condition = \
    (data["numPhase"][j,:,:] != 0) & \
    (data["numPhase"][j,:,:] != 2) & \
    (data["numPhase"][j,:,:] != 3) & \
    (data["sdj"][j,:,:] >= data["seuilTempPhaseSuivante"][j,:,:])

  data["changePhase"][j,:,:] = np.where(condition, 1, data["changePhase"][j,:,:])

  ### test phase 3
  condition = \
    (data["numPhase"][j,:,:] == 3) & \
    (data["phasePhotoper"][j,:,:] == 0)

  data["changePhase"][j,:,:] = np.where(condition, 1, data["changePhase"][j,:,:])

  
  
  ###### incrémentation de la phase
  condition = \
    (data["numPhase"][j,:,:] != 0) & \
    (data["changePhase"][j,:,:] == 1) & \
    (data["initPhase"][j,:,:] != 1)  

  data["numPhase"][j:,:,:] = np.where(
    condition,
    data["numPhase"][j,:,:] + 1 ,
    data["numPhase"][j,:,:],
  )#[np.newaxis,...]

  # on enregistre les sdj de la phase précédente
  data["sommeDegresJourPhasePrec"][j:,:,:] = np.where(
      condition,
      data["seuilTempPhaseSuivante"][j,:,:],
      data["sommeDegresJourPhasePrec"][j,:,:],
  )[np.newaxis,...]



  ### on met à jour les températures de changement de phase
  # phase 1
  condition = \
    (data["numPhase"][j,:,:] == 1) & \
    (data["changePhase"][j,:,:] == 1)

  data["seuilTempPhaseSuivante"][j:,:,:] = np.where(
      condition,
      paramVariete["SDJLevee"],
      data["seuilTempPhaseSuivante"][j,:,:]
  )[np.newaxis,...]

  # phase 2
  condition = \
    (data["numPhase"][j,:,:] == 2) & \
    (data["changePhase"][j,:,:] == 1)

  data["seuilTempPhaseSuivante"][j:,:,:] = np.where(
      condition,
      data["seuilTempPhaseSuivante"][j,:,:] + paramVariete["SDJBVP"],
      data["seuilTempPhaseSuivante"][j,:,:]
  )[np.newaxis,...]

  # phase 3
  condition = \
    (data["numPhase"][j,:,:] == 3) & \
    (data["changePhase"][j,:,:] == 1)

  data["phasePhotoper"][j,:,:] = np.where(
      condition,
      1,
      data["phasePhotoper"][j,:,:],
  )  

  # phase 4
  condition = \
    (data["numPhase"][j,:,:] == 4) & \
    (data["changePhase"][j,:,:] == 1)

  data["seuilTempPhaseSuivante"][j:,:,:] = np.where(
      condition,
      data["sdj"][j,:,:] + paramVariete["SDJRPR"],
      data["seuilTempPhaseSuivante"][j,:,:]
  )[np.newaxis,...]

  # phase 5
  condition = \
    (data["numPhase"][j,:,:] == 5) & \
    (data["changePhase"][j,:,:] == 1)

  data["seuilTempPhaseSuivante"][j:,:,:] = np.where(
      condition,
      data["seuilTempPhaseSuivante"][j,:,:] + paramVariete["SDJMatu1"],
      data["seuilTempPhaseSuivante"][j,:,:]
  )[np.newaxis,...]

  # phase 6
  condition = \
    (data["numPhase"][j,:,:] == 6) & \
    (data["changePhase"][j,:,:] == 1)

  data["seuilTempPhaseSuivante"][j:,:,:] = np.where(
      condition,
      data["seuilTempPhaseSuivante"][j,:,:] + paramVariete["SDJMatu2"],
      data["seuilTempPhaseSuivante"][j,:,:]
  )[np.newaxis,...]                                                    




  return data

## evalFESW

In [ ]:
def EvalFESW(j, data):
    """
    depuis bileau.pas

    Estimation de la fraction d'eau evaporable, rapporte donc au reservoir
    de surface, RuSurf est le stock d'eau maxi disponible pour la plante
    sur ce reservoir
    Modif : on considere que pour l'�vaporation la moitie de cette
    valeur doit etre ajout�e.
    // Parametres
    IN:
    StRusurf : mm
    RuSurf : mm
    OUT:
    fesw : mm
    """

    #! renaming stRuSurf to surface_tank_stock
    #! renaming ruSurf with surface_tank_capacity
    #// data["fesw"][j,:,:] = data["stRuSurf"][j,:,:] / (data["ruSurf"][j,:,:] + data["ruSurf"][j,:,:] / 10)
    data["fesw"][j,:,:] = data["surface_tank_stock"][j,:,:] / (data["surface_tank_capacity"][j,:,:] + data["surface_tank_capacity"][j,:,:] / 10)

    return data

## EvalKceMc

In [ ]:
def EvalKceMc(j, data, paramITK):
    """
    depuis bileau.pas
    
    Trois possibilit�s d'extinction sur l'�vaporation :
    ltr : couverture de la plante
    Mulch : effet couvrant permanent et constant; 100 pas de Mulch, 0 couvert complet bache)
    exp() �quivalent � formule de calcul du ltr mais appliqu� � l'effet couvrant d'un
    mulch couvert de paillis... �volutif    
    """

    # Kce := Mulch/100 * ltr * exp(-coefMc * SurfMc * BiomMc/1000);
    data["kce"][j,:,:] = paramITK["mulch"] / 100 * data["ltr"][j,:,:] * np.exp(-paramITK["coefMc"] * paramITK["surfMc"] * data["biomMc"][j,:,:]/1000)
    
    return data

## DemandeSol

In [ ]:
def DemandeSol(j, data):
    """
    depuis bileau.pas

    Estimation de l'evaporation potentielle du sol, on ne tient pas
    compte d'une variation de l'evaporation en fonction d'une humectation
    differente entre le haut et le bas du reservoir, on a un parametre
    mulch qui peu traduire le phenomene d'auto mulching (defaut : 0.7)
    qui peu aussi traduire un mulch par couverture vegetale ou...
    La reduction de l'evaporation par l'evolution de la couverture
    du sol par la plante est traduit par ltr.

    // Parametres
    IN:
    ETo : mm
    Kce : %
    OUT:
    evapPot : mm
    """
    # group 44
    data["evapPot"][j:,:,:] = (data["ET0"][j,:,:] * data["kce"][j,:,:]).copy()[...,np.newaxis]

    return data

## EvapMc

In [ ]:
def EvapMc(j, data, paramITK):
    """
    group 47
    depuis bileau.pas

    comme pour FESW on retire du stock la fraction evaporable
    la demande climatique étant réduite é la fraction touchant le sol ltr
    on borne é 0
    """
    # on doit reset FEMcW à chaque cycle ?
    # Var FEMcW : double;

    # group 45
    data["FEMcW"][j,:,:] = np.where(
        data["stockMc"][j,:,:] > 0,
        (paramITK["humSatMc"] * data["biomMc"][j,:,:] * 0.001) / data["stockMc"][j,:,:],
        data["FEMcW"][j,:,:],
    )

    # group 46
    data["stockMc"][j:,:,:] = np.maximum(
        0,
        data["stockMc"][j,:,:] - data["ltr"][j,:,:] * data["ET0"][j,:,:] * data["FEMcW"][j,:,:]**2,
    )[...,np.newaxis]

    return data

## EvapRuSurf

In [ ]:
def EvapRuSurf(j, data):
    """
    group 48
    depuis bileau.pas 

    Estimation de l'evaporation relle, rapporte a la fraction d'eau evaporable
    // Parametres
    IN:
    fesw : mm
    evapPot : mm
    stRuSurf : mm
    OUT:
    evap : mm
    """
    #! replacing stRuSurf by surface_tank_stock
    #// data["evap"][j:,:,:] = np.minimum(data["evapPot"][j,:,:] * data["fesw"][j,:,:]**2, data["stRuSurf"][j,:,:])[...,np.newaxis]
    data["evap"][j:,:,:] = np.minimum(data["evapPot"][j,:,:] * data["fesw"][j,:,:]**2, data["surface_tank_stock"][j,:,:])[...,np.newaxis]

    return data

## EvalFTSW

In [ ]:
def EvalFTSW(j, data):
    """
    group 49
    depuis bileau.pas 

    Estimation de la fraction d'eau transpirable, rapporte donc au reservoir
    contenant les racines
    // Parametres
    IN:
    RuRac : mm
    StockRac : mm
    OUT:
    ftsw : mm
    """

    data["ftsw"][j:,:,:] = np.where(
        #! renaming stRurMax to root_tank_capacity
        #// data["stRurMax"][j,:,:] > 0,
        data["root_tank_capacity"][j,:,:] > 0,
        #! renaming stRur to root_tank_stock
        #! renaming stRurMax to root_tank_capacity
        #// data["stRur"][j,:,:] / data["stRurMax"][j,:,:],
        data["root_tank_stock"][j,:,:] / data["root_tank_capacity"][j,:,:],
        0,
    )[...,np.newaxis]

    return data

## EvolKcpKcIni

In [ ]:
def EvolKcpKcIni(j, data, paramVariete):
    # group 50
    
    # //manque   Numphase := trunc(NoPhase); ??
    # d'après biomasse.pas : 
    # kcp := max ( 0.3,KcMax * (1 - Ltr));

    data["kcp"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] >= 1,
        np.maximum(0.3, paramVariete["kcMax"] * (1 - data["ltr"][j,:,:])),
        data["kcp"][j,:,:],
    )[...,np.newaxis]
    
    return data

## DemandePlante

In [ ]:
def DemandePlante(j, data):
    # ggroup 51
    # d'près bileau.pas
    # TrPot := Kcp * ETo;
    # attention, séparation de ETp et ET0 dans les formules
    data["trPot"][j:,:,:] = (data["kcp"][j,:,:] * data["ET0"][j,:,:]).copy()[...,np.newaxis]
    
    return data

## EvalKcTot

In [ ]:
def EvalKcTot(j, data):
    # group 52
    # d'après bileau.pas
    # added a condition on 19/08/22 to match SARRA-H original behavior
    data["kcTot"][j:,:,:] = np.where(
        data["kcp"][j,:,:] == 0.0,
        data["kce"][j,:,:],
        data["kce"][j,:,:] + data["kcp"][j,:,:],
    )[...,np.newaxis]

    return data

## CstrPFactor

In [ ]:
def CstrPFactor(j, data, paramVariete):
    # group 57
    # d'après bileau.pas

    # group 53
    data["pFact"][j:,:,:] = paramVariete["PFactor"] + 0.04 * (5 - np.maximum(data["kcp"][j,:,:], 1) * data["ET0"][j,:,:])[...,np.newaxis]

    # group 54
    data["pFact"][j:,:,:] = np.minimum(np.maximum(0.1, data["pFact"][j,:,:]), 0.8)[...,np.newaxis]

    #group 55
    data["cstr"][j:,:,:] = np.minimum((data["ftsw"][j,:,:] / (1 - data["pFact"][j,:,:])), 1)[...,np.newaxis]

    # group 56
    data["cstr"][j:,:,:] = np.maximum(0, data["cstr"][j,:,:])[...,np.newaxis]

    return data

## EvalTranspi

In [ ]:
def EvalTranspi(j, data):
    # d'après bileau.pas
    # group 58
    data["tr"][j:,:,:] = (data["trPot"][j,:,:] * data["cstr"][j,:,:]).copy()[...,np.newaxis]
    return data

## ConsoResSep

In [ ]:
def ConsoResSep(j, data):
    """
    d'après bileau.pas

    group 71

    Separation de tr et evap. Consommation de l'eau sur les reservoirs
    Hypothese : l'evaporation est le processus le plus rapide, retranche
    en premier sur le reservoir de surface. Comme reservoir de surface
    et reservoirs racinaires se chevauchent, il nous faut aussi calcule sur
    le reservoir ayant des racines la part deja extraite pour l'evaporation.
    Quand la profondeur des racines est inferieur au reservoir de surface
    on ne consomme en evaporation que la fraction correspondant a cette
    profondeur sur celle du reservoir de surface (consoRur).
    Les estimations d'evaporation et de transpirations sont effectues
    separemment, on peut ainsi avoir une consommation legerement superieure
    a l'eau disponible. On diminuera donc la transpiration en consequence.

    Modif : Pour les stock d'eau on tient compte de la partie rajoutee au
    reservoir de surface qui ne peut etre que evapore (air dry)
    // Parametres
    IN:
    stRurMax : mm
    RuSurf : mm
    evap : mm
    trPot : mm
    evaPot : mm
    INOUT :
    stRuSurf : mm
    tr : mm
    stRur : mm
    stRu : mm
    OUT:
    etr : mm
    etm : mm
    """

    # part transpirable sur le reservoir de surface
    # group 59
    #! replacing stRuSurf by surface_tank_stock
    #! renaming ruSurf with surface_tank_capacity
    #// data["trSurf"][j:,:,:] = np.maximum(0, data["stRuSurf"][j,:,:] - data["ruSurf"][j,:,:] / 10)[...,np.newaxis]
    data["trSurf"][j:,:,:] = np.maximum(0, data["surface_tank_stock"][j,:,:] - data["surface_tank_capacity"][j,:,:] / 10)[...,np.newaxis]

    # qte d'eau evapore a consommer sur le reservoir de surface
    # group 60
    #! replacing stRuSurf by surface_tank_stock
    #// data["stRuSurf"][j:,:,:] = np.maximum(0, data["stRuSurf"][j,:,:] - data["evap"][j,:,:])[...,np.newaxis]
    data["surface_tank_stock"][j:,:,:] = np.maximum(0, data["surface_tank_stock"][j,:,:] - data["evap"][j,:,:])[...,np.newaxis]


    # qte d'eau evapore a retirer sur la part transpirable
    # group 61
    data["consoRur"][j:,:,:] = np.where(
        data["evap"][j,:,:] > data["trSurf"][j,:,:],
        data["trSurf"][j,:,:],
        data["evap"][j,:,:],
    )[...,np.newaxis]

    # data["stRu"][j:,:,:] = np.maximum(0, data["stRu"][j,:,:] - data["consoRur"][j,:,:])
    # essais stTot
    # group 62
    #! renaming stTot to total_tank_stock
    #// data["stTot"][j:,:,:] = np.maximum(0, data["stTot"][j,:,:] - data["consoRur"][j,:,:])[...,np.newaxis]
    data["total_tank_stock"][j:,:,:] = np.maximum(0, data["total_tank_stock"][j,:,:] - data["consoRur"][j,:,:])[...,np.newaxis]

    #  fraction d'eau evapore sur la part transpirable qd les racines sont moins
    #  profondes que le reservoir de surface, mise a jour des stocks transpirables
    # group 63
    data["consoRur"][j:,:,:] = np.where(
        #! renaming stRurMax with root_tank_capacity
        #! renaming ruSurf with surface_tank_capacity
        #// data["stRurMax"][j,:,:] < data["ruSurf"][j,:,:],
        data["root_tank_capacity"][j,:,:] < data["surface_tank_capacity"][j,:,:],
        #! renaming stRur to root_tank_stock
        #! renaming ruSurf with surface_tank_capacity
        #// data["evap"][j,:,:] * data["stRur"][j,:,:] / data["ruSurf"][j,:,:],
        data["evap"][j,:,:] * data["root_tank_stock"][j,:,:] / data["surface_tank_capacity"][j,:,:],
        data["consoRur"][j,:,:],
    )[...,np.newaxis]

    # group 64
    #! renaming stRur to root_tank_stock
    #// data["stRur"][j:,:,:] = np.maximum(0, data["stRur"][j,:,:] - data["consoRur"][j,:,:])[...,np.newaxis]
    data["root_tank_stock"][j:,:,:] = np.maximum(0, data["root_tank_stock"][j,:,:] - data["consoRur"][j,:,:])[...,np.newaxis]


    # // reajustement de la qte transpirable considerant que l'evap a eu lieu avant
    # // mise a jour des stocks transpirables  
    # group 65
    data["tr"][j:,:,:] = np.where(
        #! renaming stRur to root_tank_stock
        #// data["tr"][j,:,:] > data["stRur"][j,:,:],
        data["tr"][j,:,:] > data["root_tank_stock"][j,:,:],
        #// np.maximum(data["stRur"][j,:,:] - data["tr"][j,:,:], 0),
        np.maximum(data["root_tank_stock"][j,:,:] - data["tr"][j,:,:], 0),
        data["tr"][j,:,:],
    )[...,np.newaxis]


    # group 66
    #! renaming stRuSurf with surface_tank_stock
    #// data["stRuSurf"][j:,:,:] = np.where(
    data["surface_tank_stock"][j:,:,:] = np.where(
        #! renaming stRur to surface_tank_stock
        #// data["stRur"][j,:,:] > 0,
        data["root_tank_stock"][j,:,:] > 0,
        #// np.maximum(data["stRuSurf"][j,:,:] - (data["tr"][j,:,:] * np.minimum(data["trSurf"][j,:,:]/data["stRur"][j,:,:], 1)), 0),
        #! renaming stRuSurf with surface_tank_stock
        #// np.maximum(data["stRuSurf"][j,:,:] - (data["tr"][j,:,:] * np.minimum(data["trSurf"][j,:,:]/data["root_tank_stock"][j,:,:], 1)), 0),
        np.maximum(data["surface_tank_stock"][j,:,:] - (data["tr"][j,:,:] * np.minimum(data["trSurf"][j,:,:]/data["root_tank_stock"][j,:,:], 1)), 0),
        #// data["stRuSurf"][j,:,:],
        data["surface_tank_stock"][j,:,:],
    )[...,np.newaxis]


    # group 67
    #! renaming stRur to root_tank_stock
    #// data["stRur"][j:,:,:] = np.maximum(0, data["stRur"][j,:,:] - data["tr"][j,:,:])[...,np.newaxis]
    data["root_tank_stock"][j:,:,:] = np.maximum(0, data["root_tank_stock"][j,:,:] - data["tr"][j,:,:])[...,np.newaxis]

    # data["stRu"][j:,:,:] = np.maximum(0, data["stRu"][j,:,:] - data["tr"][j,:,:])
    # essais stTot
    # group 68
    #! renaming stTot to total_tank_stock
    #// data["stTot"][j:,:,:] = np.maximum(0, data["stTot"][j,:,:] - data["tr"][j,:,:])[...,np.newaxis] 
    data["total_tank_stock"][j:,:,:] = np.maximum(0, data["total_tank_stock"][j,:,:] - data["tr"][j,:,:])[...,np.newaxis] ## ok

    # group 69
    data["etr"][j:,:,:] = (data["tr"][j,:,:] + data["evap"][j,:,:]).copy()[...,np.newaxis]
    
    # group 70
    data["etm"][j:,:,:] = (data["trPot"][j,:,:] + data["evapPot"][j,:,:]).copy()[...,np.newaxis]

    return data

## EvalVitesseRacSarraV3

In [ ]:
def EvalVitesseRacSarraV3(j, data, paramVariete):
    # d'après phenologie.pas
    # group 79

    # EvalVitesseRacSarraV3

    # phase 1
    #group 72
    data["vRac"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 1,
        paramVariete['VRacLevee'],
        data["vRac"][j,:,:],
    )[...,np.newaxis]

    # phase 2
    # group 73
    data["vRac"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 2,
        paramVariete['VRacBVP'],
        data["vRac"][j,:,:],
    )[...,np.newaxis]

    # phase 3
    # group 74
    data["vRac"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 3,
        paramVariete['VRacPSP'],
        data["vRac"][j,:,:],
    )[...,np.newaxis]

    # phase 4
    # group 75
    data["vRac"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 4,
        paramVariete['VRacRPR'],
        data["vRac"][j,:,:],
    )[...,np.newaxis]

    # phase 5
    # group 76
    data["vRac"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 5,
        paramVariete['VRacMatu1'],
        data["vRac"][j,:,:],
    )[...,np.newaxis]
    
    # phase 6
    # group 77
    data["vRac"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 6,
        paramVariete['VRacMatu2'],
        data["vRac"][j,:,:],
    )[...,np.newaxis]

    # phase 0 ou 7
    #     else
    #  VitesseRacinaire := 0
    #
    # group 78
    data["vRac"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] == 0) | (data["numPhase"][j,:,:] == 7),
        0,
        data["vRac"][j,:,:],
    )[...,np.newaxis]

    return data

## EvalLtr

In [ ]:
def EvalLtr(j, data, paramVariete):
    # group 80
    # d'après biomasse.pas 

    
    # ltr : Taux de rayonnement transmis au sol. Unités : MJ/MJ
    data["ltr"][j:,:,:] = np.exp(-paramVariete["kdf"] * data["lai"][j,:,:])[...,np.newaxis]
    
    return data

## EvalConversion

In [ ]:
def EvalConversion(j, data, paramVariete):
    # d'après milbilancarbone.pas 
    # group 87

    # EvalConversion
    # group 81
    data["KAssim"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 2,
        1,
        data["KAssim"][j,:,:],
    )[...,np.newaxis]

    # group 82
    data["KAssim"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 3,
        paramVariete['txAssimBVP'],
        data["KAssim"][j,:,:],
    )[...,np.newaxis]

    # group 83
    data["KAssim"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 4,
        paramVariete['txAssimBVP'],
        data["KAssim"][j,:,:],
    )[...,np.newaxis]

    # group 84
    data["KAssim"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 5,
        paramVariete["txAssimBVP"] + (data['sdj'][j,:,:] - data['sommeDegresJourPhasePrec'][j,:,:]) * (paramVariete['txAssimMatu1'] -  paramVariete['txAssimBVP']) / (data['seuilTempPhaseSuivante'][j,:,:] - data['sommeDegresJourPhasePrec'][j,:,:]),
        data["KAssim"][j,:,:],
    )[...,np.newaxis]

    # group 85
    data["KAssim"][j:,:,:] = np.where(
        data["numPhase"][j,:,:] == 6,
        paramVariete["txAssimMatu1"] + (data["sdj"][j,:,:] - data["sommeDegresJourPhasePrec"][j,:,:]) * (paramVariete["txAssimMatu2"] - paramVariete["txAssimMatu1"]) / (data["seuilTempPhaseSuivante"][j,:,:] - data["sommeDegresJourPhasePrec"][j,:,:]),
        data["KAssim"][j,:,:],
    )[...,np.newaxis]

    # group 86
    data["conv"][j:,:,:] = (data["KAssim"][j,:,:] * paramVariete["txConversion"])[...,np.newaxis] # Conversion:=KAssim*EpsiB;

    return data

## BiomDensOptSarV42

In [ ]:
def BiomDensOptSarV42(j, data, paramITK, paramVariete):
    """
    d'après bilancarbonsarra.pas

    { si densit� plus faible alors on consid�re qu'il faut augmenter les biomasses, LAI etc
    en regard de cette situation au niveau de chaque plante (car tout est rapport� � des kg/ha).
    Si elle est plus forte elle augmente de fa�on asymptotique.
    }

    """

    if ~np.isnan(paramVariete["densOpti"]) :

        # group 88
        data["rapDensite"] = paramVariete["densiteA"] + paramVariete["densiteP"] * np.exp(-(paramITK["densite"] / ( paramVariete["densOpti"]/- np.log((1 - paramVariete['densiteA'])/ paramVariete["densiteP"]))))
        
        # group 89
        data["rdt"][j:,:,:] = (data["rdt"][j,:,:] * data["rapDensite"])[...,np.newaxis]

        # group 90
        data["rdtPot"][j:,:,:] = (data["rdtPot"][j,:,:] * data["rapDensite"])[...,np.newaxis]

        # group 91
        data["biomasseRacinaire"][j:,:,:] = (data["biomasseRacinaire"][j,:,:] * data["rapDensite"])[...,np.newaxis]
        # group 92
        data["biomasseTige"][j:,:,:] = (data["biomasseTige"][j,:,:] * data["rapDensite"])[...,np.newaxis]
        # group 93
        data["biomasseFeuille"][j:,:,:] = (data["biomasseFeuille"][j,:,:] * data["rapDensite"])[...,np.newaxis]

        # group 94
        data["biomasseAerienne"][j:,:,:] = (data["biomasseTige"][j,:,:] + data["biomasseFeuille"][j,:,:] + data["rdt"][j,:,:])[...,np.newaxis]
        #data["biomasseAerienne"][j:,:,:] = data["biomasseAerienne"][j,:,:] * data["rapDensite"]
        
        # group 95
        data["lai"][j:,:,:]  = (data["biomasseFeuille"][j,:,:] * data["sla"][j,:,:])[...,np.newaxis]
        #data["lai"][j:,:,:]  = data["lai"][j:,:,:]  * data["rapDensite"]

        # group 96
        data["biomasseTotale"][j:,:,:] = (data["biomasseAerienne"][j,:,:] + data["biomasseRacinaire"][j,:,:])[...,np.newaxis]
        #data["biomasseTotale"][j:,:,:] = data["biomasseTotale"][j:,:,:] * data["rapDensite"]
    
    return data

## EvalAssimSarrahV42

In [ ]:
def EvalAssimSarrahV42(j, data, paramITK, paramVariete):

    """
    group 100
    d'après bilancarbonsarra.pas 

    Modif du 04/03/2021 : Prise en compte en plus de la densit� de semis de l'effet niveau d'intensification NI
    NI = 1 quand on est � l'optimum du niveau d'intensification. Dans le cas de situation contr�l� c'est
    la fertilit� qui est la clef principale en prenant en r�f�rence la qt� d'azote (�quivalent phosphore...) optimum
    Il peut aller � 0 ou �tre sup�rieur � 1 si situation sur optimum, ie un peu plus de rdt mais � cout trop �lev�...
    On �value un nouveau tx de conversion en fn du Ni au travers d'une double �quation : asympote x gaussienne invers�e
    Et d'un NI d�fini en fn du sc�nario de simulation ou des donn�es observ�es.
    NIYo = D�calage en Y de l'asymptote
    NIp  = pente de l'asymptote
    LGauss = Largeur de la Guaussienne
    AGauss = Amplitude de la Guaussienne

    Conversion qui est la valeur du taux de conversion en situation optimum n'a plus besoin d'�tre utilis� sinon
    dans la calibration des param�tres de cette �quation en absence de donn�es sur ces param�tres on ne met aucune valeur � NI
    CF fichier ex IndIntensite_txConv_eq.xls}

    """
    # on rajoute le parIntercepte depuis evalassimsarrahv4
    # data["parIntercepte"][j,:,:] = 0.5 * (1 - data["ltr"][j,:,:]) * data["rg"][j,:,:]

    # group 97
    if ~np.isnan(paramITK["NI"]): 
        #correction des taux de conversion en fonction des niveaux d'intensification
        # paramVariete["txConversion"] = paramVariete["NIYo"] + paramVariete["NIp"] * (1-np.exp(-paramVariete["NIp"] * paramITK["NI"])) - (np.exp(-0.5*((paramITK["NI"] - paramVariete["LGauss"])/paramVariete["AGauss"])* (paramITK["NI"]- paramVariete["LGauss"])/paramVariete["AGauss"]))/(paramVariete["AGauss"]*2.506628274631)

        data["assimPot"][j,:,:] = data["par"][j,:,:] * (1-np.exp(-paramVariete["kdf"] * data["lai"][j,:,:])) * paramVariete["txConversion"] * 10

    else :

        data["assimPot"][j,:,:] = data["par"][j,:,:] * (1-np.exp(-paramVariete["kdf"] * data["lai"][j,:,:])) * data["conv"][j,:,:] * 10

    # group 98
    data["assim"][j,:,:] = np.where(
        data["trPot"][j,:,:] > 0,
        data["assimPot"][j,:,:] * data["tr"][j,:,:] / data["trPot"][j,:,:],
        0,
    )


    return data

## EvalRespMaintSarrahV3

In [ ]:
def EvalRespMaintSarrahV3(j, data, paramVariete):
    """
    # group 103
    d'après bilancarbonsarra.pas
    
    RespMaint Kg/ha/j  en équivalent matiére séche
    KRespMaint     (0.015)
    KTempMaint éC  (25 )
    """

    # group 101
    # on cast sur j
    # kRespMaint = txRespMaint ?
    # dRespMaint = respMaint ?
    data["respMaint"][j,:,:] =  ((paramVariete["kRespMaint"] * data["biomasseTotale"][j,:,:] * (2**((data["tpMoy"][j,:,:] - paramVariete["tempMaint"]) / 10)))) + (paramVariete["kRespMaint"] * data["biomasseFeuille"][j,:,:] * (2**((data["tpMoy"][j,:,:] -  paramVariete["tempMaint"]) / 10)))

    # group 102
    # Question pourquoi > 4 tous le s !!! si pas de feuilles mort !!! 
    # # on cast sur j	
    data["respMaint"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] > 4) & (data["biomasseFeuille"][j,:,:]==0),
        0,
        data["respMaint"][j,:,:],
    )[...,np.newaxis]

    return data

## EvolBiomTotSarrahV4

In [ ]:
def EvolBiomTotSarrahV4(j, data, paramVariete, paramITK):
    """
    groupo 106
    d'après bilancarbonsarra.pas
    {
    On harmonise avec une biomasse rapportée é une plante
    et pas par hectare pour l'initialisation de la biomasse é l'émergenceé phase 2
    }
    """

    ##! attention au pb de densité de semis à corriger

        #biomasse totale en kg/ha
    # on initialise en date du passage à la phase 2,
    # sino, biomasse totale fonction de la biomasse totale t-1 

    # BiomasseTotale := Densite* Max(1,70000/Densite) * KResGrain * BiomasseGrain / 1000;
    
    # group 104
    data["biomasseTotale"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:]==2) & (data["changePhase"][j,:,:]==1),
        paramITK["densite"] *  np.maximum(1,paramVariete['densOpti']/paramITK['densite']) * paramVariete["txResGrain"] *  paramVariete["poidsSecGrain"] / 1000,
        # paramITK["densite"] *  np.maximum(1,70000/paramITK['densite']) * paramVariete["txResGrain"] *  paramVariete["poidsSecGrain"] / 1000,
        data["biomasseTotale"][j,:,:]  + (data["assim"][j,:,:] - data["respMaint"][j,:,:]),
    )[...,np.newaxis]

    

    # on cast sur j
    data["deltaBiomasseTotale"][j:,:,:] = (data["assim"][j,:,:] - data["respMaint"][j,:,:]).copy()[...,np.newaxis]

    return data

## EvalRdtPotRespSarV42

In [ ]:
def EvalRdtPotRespSarV42(j, data, paramVariete):
    # group 111
    # d'après bilancarbonsarra.pas

    #! biomTotStadeIp laissé à 0 ?
    # data["biomTotStadeIp"][j:,:,:] = np.where(
    #     (data["numPhase"][j,:,:] == 4) & (data["changePhase"][j,:,:] == 1),
    #     data["biomasseTotale"][j,:,:],
    #     data["biomTotStadeIp"][j,:,:],
    # )

    # group 107
    data["biomTotStadeFloraison"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] == 5) & (data["changePhase"][j,:,:] == 1),
        data["biomasseTotale"][j,:,:],
        data["biomTotStadeFloraison"][j,:,:],
    )[...,np.newaxis]


    #group 108
    data["rdtPot"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] == 5) & (data["changePhase"][j,:,:] == 1),
        (paramVariete["KRdtPotA"] * (data["biomTotStadeFloraison"][j,:,:] - data["biomTotStadeIp"][j,:,:]) + paramVariete["KRdtPotB"]) + paramVariete["KRdtBiom"] * data["biomTotStadeFloraison"][j,:,:],
        data["rdtPot"][j,:,:],
    )[...,np.newaxis]

    #group 109
    #! phaseDevVeg pas utilisé ? attention c'est un paramètre variétal et pas un jeu de donées
    data["rdtPot"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] == 5) & (data["changePhase"][j,:,:] == 1) & (data["rdtPot"][j,:,:] > data["biomasseTige"][j,:,:] * 2) & (data["phaseDevVeg"][j,:,:] < 6),
        data["biomasseTige"][j,:,:] * 2,
        data["rdtPot"][j,:,:],
    )[...,np.newaxis]

    # group 110
    # dRdtPot = rdtPotDuJour
    data["dRdtPot"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] == 5),
        np.where(
            (data["trPot"][j,:,:]>0),
            np.maximum(
                #! pourquoi est ce que c'est un rapport de ddj sur sdj, et pas sdj sur sdj ?
                data["rdtPot"][j,:,:] * (data["ddj"][j,:,:] / paramVariete["SDJMatu1"]) * (data["tr"][j,:,:] / data["trPot"][j,:,:]),
                data["respMaint"][j,:,:] * 0.15,
            ),
            0,
        ),
        data["dRdtPot"][j,:,:],
    )[...,np.newaxis]

    
    return data

## EvolBiomAeroSarrahV3

In [ ]:

def EvolBiomAeroSarrahV3(j, data, paramVariete):
    # group 115
    #  verif vs code pascal OK
    # d'après bilancarbonesarra.pas

    # on stocke la valeur précédente dans deltaBiomasseAerienne
    # deltabiomasseaerienne est juste la différence entre j et j-1
    # group 112
    data["deltaBiomasseAerienne"][j:,:,:] = np.copy(data["biomasseAerienne"][j,:,:])[...,np.newaxis]

    # la biomasseAerienne est égale, sur les stades 2 à 4, à un coeff borné au max en 0.9
    # fois la biomasse totale
    # en dehors de ces stades, elle vaut la différence entre la biomasse totale
    # et la biomasse aerienne
    # on étend sur j
    #group 113
    data["biomasseAerienne"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] >= 2) & (data["numPhase"][j,:,:] <= 4),
        np.minimum(0.9, paramVariete["aeroTotPente"] * data["biomasseTotale"][j,:,:] + paramVariete["aeroTotBase"]) * data["biomasseTotale"][j,:,:],
        data["biomasseAerienne"][j,:,:] + data["deltaBiomasseTotale"][j,:,:],
    )[...,np.newaxis]




    # version ocelet
    # data["deltaBiomasseAerienne"][j:,:,:] = (data["biomasseAerienne"][j,:,:] - data["deltaBiomasseArienne"][j,:,:])
    # version originale
    # group 114
    data["deltaBiomasseAerienne"][j:,:,:] = (data["biomasseAerienne"][j,:,:] - data["deltaBiomasseAerienne"][j,:,:])[...,np.newaxis]
    # version modifiée
    # data["deltaBiomasseAerienne"][j,:,:] = data["biomasseAerienne"][j,:,:] - data["biomasseAerienne"][j,:,:-1]


    
    
    return data

## EvalReallocationSarrahV3

In [ ]:
def EvalReallocationSarrahV3(j, data, paramVariete):
    """
    groupe 118
    d'après bilancarbonesarra.pas
    {
    La reallocation est é 0 quand la biomasse foliaire verte est inférieure é 30 kh/ha
    }
    """

    condition = (data["numPhase"][j,:,:] == 5)
        # on cast sur j
        # group 116
    data["manqueAssim"][j:,:,:] = np.where(
        condition,
        np.maximum(0, (data["dRdtPot"][j,:,:] -  np.maximum(0.0, data["deltaBiomasseAerienne"][j,:,:]))),
        0,
    )[...,np.newaxis]
    # on cast sur j

    # group 117
    data["reallocation"][j:,:,:] = np.where(
        condition,
        np.minimum(data["manqueAssim"][j,:,:] *  paramVariete["txRealloc"],  np.maximum(0.0, data["biomasseFeuille"][j,:,:] - 30)),
        0,
    )[...,np.newaxis]

    return data

## EvalBiomasseRacinaire

In [ ]:
def EvalBiomasseRacinaire(j, data):
    #d'après milbilancarbone.pas
    # groupe 119
    data["biomasseRacinaire"][j,:,:] = data["biomasseTotale"][j,:,:] - data["biomasseAerienne"][j,:,:]

    return data

## EvalFeuilleTigeSarrahV4

In [ ]:
def EvalFeuilleTigeSarrahV4(j, data, paramVariete):

    # d'après bilancarbonsarra.pas

    # on reset bM et CM en début de procédure

    # si numPhase > 1
    # group 120
    data["deltaBiomasseFeuilles"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] > 1),
        data["biomasseFeuille"][j,:,:],
        data["deltaBiomasseFeuilles"][j,:,:],
    )[...,np.newaxis]


    #     {
    # Modif du 15/06/2020     Je ne sais pourquoi max(10 avant max(O � v�rifier si division par BiomasseFeuilles?
    # }
    # si deltaBiomasseAerienne < 0
    condition = (data["numPhase"][j,:,:] > 1) & \
        (data["deltaBiomasseAerienne"][j,:,:] < 0)

    # group 121
    data["biomasseFeuille"][j:,:,:] = np.where(
        condition,
        np.maximum(0.00000001, data["biomasseFeuille"][j,:,:] - (- data["deltaBiomasseAerienne"][j,:,:] + data["reallocation"][j,:,:]) \
             * paramVariete["pcReallocFeuille"]),
        data["biomasseFeuille"][j,:,:],
    )[...,np.newaxis]

    # group 122
    data["biomasseTige"][j:,:,:] = np.where(
        condition,
        np.maximum(0.00000001, data["biomasseTige"][j,:,:] - (- data["deltaBiomasseAerienne"][j,:,:] + data["reallocation"][j,:,:]) \
            * (1 - paramVariete["pcReallocFeuille"])),
        data["biomasseTige"][j,:,:],
    )[...,np.newaxis]



    # si deltaBiomasseAerienne >= 0
    condition = (data["numPhase"][j,:,:] > 1) & \
        (data["deltaBiomasseAerienne"][j,:,:] >= 0) & \
        ((data["numPhase"][j,:,:] <= 4) | (data["numPhase"][j,:,:] <= paramVariete["phaseDevVeg"]))
        # (data["numPhase"][j,:,:] <= 4)



    # group 123
    data["bM"][j,:,:] = np.where(
        condition,
        paramVariete["feuilAeroBase"] - 0.1,
        data["bM"][j,:,:],
    )


    # group 124
    data["cM"][j,:,:] = np.where(
        condition,
        ((paramVariete["feuilAeroPente"] * 1000)/ data["bM"][j,:,:] + 0.78) / 0.75,
        data["cM"][j,:,:],
    )


    # group 125
    data["biomasseFeuille"][j:,:,:] = np.where(
        condition,
        (0.1 + data["bM"][j,:,:] * data["cM"][j,:,:] ** ((data["biomasseAerienne"][j,:,:] - data["rdt"][j,:,:]) / 1000)) \
             * (data["biomasseAerienne"][j,:,:] - data["rdt"][j,:,:]),
        data["biomasseFeuille"][j,:,:],
    )[...,np.newaxis]




    # group 126
    data["biomasseTige"][j:,:,:] = np.where(
        condition,
        data["biomasseAerienne"][j,:,:] - data["biomasseFeuille"][j,:,:] - data["rdt"][j,:,:],
        data["biomasseTige"][j,:,:],
    )[...,np.newaxis]


    
    condition = (data["numPhase"][j,:,:] > 1) & \
        (data["deltaBiomasseAerienne"][j,:,:] >= 0)

    # group 127
    data["biomasseFeuille"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] > 1) & (data["deltaBiomasseAerienne"][j,:,:] > 0),
        data["biomasseFeuille"][j,:,:] - data["reallocation"][j,:,:] * paramVariete["pcReallocFeuille"],
        data["biomasseFeuille"][j,:,:],
    )[...,np.newaxis]


    # group 128
    data["biomasseTige"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] > 1) & (data["deltaBiomasseAerienne"][j,:,:] > 0),
        data["biomasseTige"][j,:,:] - (data["reallocation"][j,:,:] * (1- paramVariete["pcReallocFeuille"])),
        data["biomasseTige"][j,:,:],
    )[...,np.newaxis]



    condition = (data["numPhase"][j,:,:] > 1) 

    # group 129
    data["deltaBiomasseFeuilles"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] > 1),
        data["biomasseFeuille"][j,:,:] - data["deltaBiomasseFeuilles"][j,:,:],
        data["deltaBiomasseFeuilles"][j,:,:],
    )[...,np.newaxis]

    #group 130
    data["biomasseAerienne"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] > 1),
        data["biomasseTige"][j,:,:] + data["biomasseFeuille"][j,:,:] + data["rdt"][j,:,:],
        data["biomasseAerienne"][j,:,:],
    )[...,np.newaxis]



    return data

## EvalBiomasseVegetati

In [ ]:
def EvalBiomasseVegetati(j, data):
    # group 132
    #d'après milbilancarbone.pas
    data["biomasseVegetative"][j:,:,:] = (data["biomasseTige"][j,:,:] + data["biomasseFeuille"][j,:,:])[...,np.newaxis]
    return data

## EvalSlaSarrahV3

In [ ]:
def EvalSlaSarrahV3(j, data, paramVariete):
    # check vs code pascal OK
    """
    groupe 136
    d'après bulancarbonsarra.pas

    On suppose que les jeunes feuilles on un SLA supérieur aux vieilles feuilles.
    La fraction de jeunes (nouvelles) feuilles fait donc monter le SLA global
    du couvert. Le paramétre penteSLA provoque une chute générale du SLA
    (penteSLA = chute relative par jour = fraction de différence entre SLAmax
    et SLAmin). Fonctionnement conéu surtout pour les légumineuses, mais
    peut étre aussi adapté aux autres espéces.
    Paramétres :
    SLAmax (0.001 é 0.01), ex : 0.007
    SLAmin (0.001 é 0.01), ex : 0.002
    penteSLA (0 é 0.2), ex : 0.1
    Avec : SLAini = SLAmax
    }
    """
    # group 133
    data["sla"][j:,:,:] = np.where(
        (data["biomasseFeuille"][j,:,:] > 0) & \
            (data["numPhase"][j,:,:] == 2) & \
            (data["changePhase"][j,:,:] == 1),
        paramVariete["slaMax"],
        data["sla"][j,:,:],
    )[...,np.newaxis]

    # Modif du 10/07/2018, DeltaBiomasse neg si reallocation ne pas fair l'evol du SLA dans ces conditions
    
    # original
    # groupe 134
    data["sla"][j:,:,:] = np.where(
        (data["biomasseFeuille"][j,:,:] > 0),
        np.where(
            (data["deltaBiomasseFeuilles"][j,:,:] > 0),
            (data["sla"][j,:,:] - paramVariete["slaPente"] * (data["sla"][j,:,:] - paramVariete["slaMin"])) \
              * (data["biomasseFeuille"][j,:,:] - data["deltaBiomasseFeuilles"][j,:,:]) / data["biomasseFeuille"][j,:,:] \
              + (paramVariete["slaMax"] + data["sla"][j,:,:])/2 * (data["deltaBiomasseFeuilles"][j,:,:] / data["biomasseFeuille"][j,:,:]),
            (data["sla"][j,:,:] - paramVariete["slaPente"] * (data["sla"][j,:,:] - paramVariete["slaMin"])) \
                * (data["biomasseFeuille"][j,:,:] / data["biomasseFeuille"][j,:,:]),
        ),
        data["sla"][j,:,:],
    )[...,np.newaxis]

    # d'après version ocelet
    # data["sla"][j:,:,:] = np.where(
    #     (data["biomasseFeuille"][j,:,:] > 0),
    #     (data["sla"][j,:,:] - paramVariete["slaPente"] * (data["sla"][j,:,:] - paramVariete["slaMin"])) \
    #         * (data["biomasseFeuille"][j,:,:] - data["deltaBiomasseFeuilles"][j,:,:]) / data["biomasseFeuille"][j,:,:] \
    #         + (paramVariete["slaMax"] + data["sla"][j,:,:])/2 * (data["deltaBiomasseFeuilles"][j,:,:] / data["biomasseFeuille"][j,:,:]),
    #     data["sla"][j,:,:],
    # )

    # mix original/ocelet
    # data["sla"][j:,:,:] = np.where(
    #     (data["biomasseFeuille"][j,:,:] > 0),
    #     np.where(
    #         (data["deltaBiomasseFeuilles"][j,:,:] > 0),
    #         (data["sla"][j,:,:] - paramVariete["slaPente"] * (data["sla"][j,:,:] - paramVariete["slaMin"])) * (data["biomasseFeuille"][j,:,:] - data["deltaBiomasseFeuilles"][j,:,:]) / data["biomasseFeuille"][j,:,:] + (paramVariete["slaMax"] + data["sla"][j,:,:])/2 * (data["deltaBiomasseFeuilles"][j,:,:] / data["biomasseFeuille"][j,:,:]),
    #         (data["sla"][j,:,:] - paramVariete["slaPente"] * (data["sla"][j,:,:] - paramVariete["slaMin"])) * (data["biomasseFeuille"][j,:,:] / data["deltaBiomasseFeuilles"][j,:,:]),
    #     ),
    #     data["sla"][j,:,:],
    # )

    # original modifié
    # data["sla"][j:,:,:] = np.where(
    #     (data["biomasseFeuille"][j,:,:] > 0),
    #     np.where(
    #         (data["deltaBiomasseFeuilles"][j,:,:] > 0),
    #         (data["sla"][j,:,:] - paramVariete["slaPente"] * (data["sla"][j,:,:] - paramVariete["slaMin"])) * (data["biomasseFeuille"][j,:,:] - data["deltaBiomasseFeuilles"][j,:,:]) / data["biomasseFeuille"][j,:,:] + (paramVariete["slaMax"] + data["sla"][j,:,:])/2 * (data["deltaBiomasseFeuilles"][j,:,:] / data["biomasseFeuille"][j,:,:]),
    #         (data["sla"][j,:,:] - paramVariete["slaPente"] * (data["sla"][j,:,:] - paramVariete["slaMin"])) * (data["biomasseFeuille"][j,:,:] / data["biomasseFeuille"][j,:,:]),
    #         # data["sla"][j,:,:],
    #     ),
    #     data["sla"][j,:,:],
    # )


    # groupe 135
    data["sla"][j:,:,:] = np.where(
        (data["biomasseFeuille"][j,:,:] > 0),
        # np.minimum(paramVariete["slaMin"], np.maximum(paramVariete["slaMax"], data["sla"][j,:,:])),
        np.minimum(paramVariete["slaMax"], np.maximum(paramVariete["slaMin"], data["sla"][j,:,:])), # d'après version ocelet
        data["sla"][j,:,:],
    )[...,np.newaxis]

    

    return data

## EvolLAIPhases

In [ ]:
def EvolLAIPhases(j, data):
    # d'après milbilancarbone.pas
    # group 137
    data["lai"][j:,:,:] = np.where(
        #(data["numPhase"][j,:,:] <= 1),
        (data["numPhase"][j,:,:] <= 1) | (data["startLock"][j,:,:] == 1),
        0,
        np.where(
            data["numPhase"][j,:,:] <= 6,
            data["biomasseFeuille"][j,:,:] * data["sla"][j,:,:],
            0,
        )
    )[...,np.newaxis]


    return data

## EvolDayRdtSarraV3

In [ ]:
def EvolDayRdtSarraV3(j, data):
    # groupe 138
    # d'après bilancarbonsarra.pas
    # {
    # On tend vers le potentiel en fn du rapport des degresJours/sumDegresJours
    # pour la phase de remplissage
    # Frein sup fn du flux de s�ve estim� par le rapport Tr/TrPot
    # }

    # dRdtPot = RdtPotDuJour
    # on cast sur j
    data["rdt"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] == 5),
        data["rdt"][j,:,:] + np.minimum(data["dRdtPot"][j,:,:],  np.maximum(0.0, data["deltaBiomasseAerienne"][j,:,:]) + data['reallocation'][j,:,:]),
        data["rdt"][j,:,:],
    )[...,np.newaxis]


    return data

## PhotoperSarrahV3

In [ ]:
def PhotoperSarrahV3(j, data, paramVariete):
    # depuis phenologie.pas
    # group 142
    """
        {Procedure speciale Vaksman Dingkuhn valable pour tous types de sensibilite
    photoperiodique et pour les varietes non photoperiodique.
    PPsens varie de 0,4 a 1,2. Pour PPsens > 2,5 = variété non photoperiodique.
    SeuilPP = 13.5
    PPcrit = 12
    SumPP est dans ce cas une variable quotidienne (et non un cumul)}
    """
    # group 139
    data["sumPP"][j,:,:] = np.where(
        (data["numPhase"][j,:,:] == 3) & (data["changePhase"][j,:,:] == 1),
        100,
        data["sumPP"][j,:,:],
    )

    # group 140
    data["sumPP"][j,:,:] = np.where(
        (data["numPhase"][j,:,:] == 3),
        (1000 / np.maximum(0.01, data["sdj"][j,:,:] - data["seuilTempPhasePrec"][j,:,:]) ** paramVariete["PPExp"]) * np.maximum(0, (data["dureeDuJour"][j,:,:] - paramVariete["PPCrit"])) / (paramVariete["SeuilPP"] - paramVariete["PPCrit"]),
        data["sumPP"][j,:,:],
    )

    # group 141
    data["phasePhotoper"][j:,:,:] = np.where(
        (data["numPhase"][j,:,:] == 3) & (data["sumPP"][j,:,:] < paramVariete["PPsens"]),
        0,
        data["phasePhotoper"][j,:,:],
    )[...,np.newaxis]

    return data

## MortaliteSarraV3

In [ ]:
def MortaliteSarraV3(j, data, paramITK, paramVariete):
    # group 150
    # test de mortalité juvénile
    #     {
    # Test sur 20 jours
    # D�s que le delta est n�gatif sur 10 jours
    # }

    condition = (data["numPhase"][j,:,:] >= 2) & (data["numPhase"][j,:,:] == 2) & (data["changePhase"][j,:,:] == 1)

    # group 143
    data['nbJourCompte'][j:,:,:] = np.where(
        condition,
        0,
        data['nbJourCompte'][j,:,:],
    )[...,np.newaxis]

    # group 144
    data['nbjStress'][j:,:,:] = np.where(
        condition,
        0,
        data['nbjStress'][j,:,:],
    )[...,np.newaxis]


    condition = (data["numPhase"][j,:,:] >= 2)

    # group 145
    data['nbJourCompte'][j:,:,:] = np.where(
        condition,
        data['nbJourCompte'][j,:,:] + 1,
        data['nbJourCompte'][j,:,:],
    )[...,np.newaxis]


    condition = (data["numPhase"][j,:,:] >= 2) & (data["nbJourCompte"][j,:,:] < paramITK["nbjTestSemis"]) & (data["deltaBiomasseAerienne"][j,:,:] < 0)

    # group 146
    data["nbjStress"][j:,:,:] = np.where(
        condition,
        data["nbjStress"][j,:,:] + 1,
        data["nbjStress"][j,:,:],                           
    )[...,np.newaxis]


    condition = (data["numPhase"][j,:,:] >= 2) & (data["nbjStress"][j,:,:] == paramVariete["seuilCstrMortality"])

    # group 147
    data["numPhase"][j,:,:] = np.where(
        condition,
        0,
        data["numPhase"][j,:,:],
    )

    # group 148
    #! renaming stRurMax with root_tank_capacity
    #// data["stRurMax"][j,:,:] = np.where(
    data["root_tank_capacity"][j,:,:] = np.where(
        condition,
        0,
        #// data["stRurMax"][j,:,:],
        data["root_tank_capacity"][j,:,:],
    )

    # group 149
    data["nbjStress"][j:,:,:] = np.where(
        condition,
        0,
        data["nbjStress"][j,:,:],
    )[...,np.newaxis]

    return data

## BiomDensiteSarraV42

In [ ]:
def BiomDensiteSarraV42(j, data, paramITK, paramVariete):
    # depuis bilancarbonsarra.pas
    #group 160
    
    if ~np.isnan(paramVariete["densOpti"]):

        #! confusion entre rapDensite dataset et parametre
        #group 151
        paramITK["rapDensite"] = paramVariete["densiteA"] + paramVariete["densiteP"] * np.exp(-(paramITK["densite"] / ( paramVariete["densOpti"]/- np.log((1 - paramVariete['densiteA'])/ paramVariete["densiteP"]))))

        # group 152
        data["rdt"][j:,:,:] = (data["rdt"][j,:,:] / data["rapDensite"])[...,np.newaxis]


        # group 153
        data["rdtPot"][j:,:,:] = (data["rdtPot"][j,:,:]/ data["rapDensite"])[...,np.newaxis]

        # group 154
        data["biomasseRacinaire"][j:,:,:] = (data["biomasseRacinaire"][j,:,:] / data["rapDensite"])[...,np.newaxis]

        # group 155
        data["biomasseTige"][j:,:,:] = (data["biomasseTige"][j,:,:] / data["rapDensite"])[...,np.newaxis]

        # group 156
        data["biomasseFeuille"][j:,:,:] = (data["biomasseFeuille"][j,:,:] / data["rapDensite"])[...,np.newaxis]

        # group 157
        data["biomasseAerienne"][j:,:,:] = (data["biomasseTige"][j,:,:] + data["biomasseFeuille"][j,:,:] + data["rdt"][j,:,:])[...,np.newaxis] 
        #data["biomasseAerienne"][j:,:,:] = data["biomasseAerienne"][j,:,:] / data["rapDensite"]


        #? conflit avec fonction evolLAIphase ?
        #  group 158
        #data["lai"][j:,:,:]  = data["biomasseFeuille"][j,:,:] * data["sla"][j,:,:]
        data["lai"][j:,:,:]  = data["lai"][j:,:,:]  / data["rapDensite"]

        # group 159
        data["biomasseTotale"][j:,:,:] = (data["biomasseAerienne"][j,:,:] + data["biomasseRacinaire"][j,:,:])[...,np.newaxis]
        #data["biomasseTotale"][j:,:,:] = data["biomasseTotale"][j:,:,:] / data["rapDensite"]






    return data


## BiomMcUBTSV3

In [ ]:
def BiomMcUBTSV3(j, data, paramITK):
    """
    depuis bilancarbonsarra.pas

    group 174

    Pendant la croissance des cultures la d�gradation des r�sidusest calcul�e sans les UBT
    Ici c'est pendant la saion s�che quand il n'y a des cultures pas de b�tes.
    Sur le mulch dress� (Up) ou couch� Lit), on calcul sa d�gradation journali�re
    sur les feuilles et les tiges en fn de coef KN (climat, termites...),
    KI ingestion par les b�tes pression en UBT seulement pour les feuilles, KT (effet pi�tinement) qui va faire passer
    du stade lev� en couch� et du stade couch� en ensevelissement pression en UBT
    Par D�faut :
    KNUp = 0.001 /jour
    KNLit = 0.011
    KN est soit une constante soit peut varier en fn climat (pas fait ref STEP)
    KT = 0.003
    KI = 0.005
    NbUBT = 10 (zone Fakara)
    """
    condition = (data["numPhase"][j,:,:] > 0)

    #   group 161
    data["UBTCulture"][j:,:,:] = np.where(condition, 0, data["NbUBT"][j,:,:])[...,np.newaxis]
    #  group 162
    data["LitFeuille"][j:,:,:] = np.where(condition, data["LitFeuille"][j,:,:] + data["FeuilleUp"][j,:,:], data["LitFeuille"][j,:,:])[...,np.newaxis]
    # group 163
    data["LitTige"][j:,:,:] = np.where(condition, data["LitTige"][j,:,:] + data["TigeUp"][j,:,:], data["LitTige"][j,:,:])[...,np.newaxis]
    # group 164
    data["FeuilleUp"][j:,:,:] = np.where(condition, 0, data["FeuilleUp"][j,:,:])[...,np.newaxis]
    # group 165
    data["TigeUp"][j:,:,:] = np.where(condition, 0, data["TigeUp"][j,:,:])[...,np.newaxis]
    # group 166
    data["biomMc"][j:,:,:] = np.where(condition, data["LitFeuille"][j,:,:] + data["LitTige"][j,:,:], data["biomMc"][j,:,:])[...,np.newaxis]

    #// D�gradation des feuilles et tiges dress�es
    # FeuilleUp := max(0, (FeuilleUp -  FeuilleUp * KNUp - FeuilleUp * KI * UBTCulture  - FeuilleUp * KT * UBTCulture));
    # group 167
    data["FeuilleUp"][j:,:,:] = np.maximum(
        0,
        data["FeuilleUp"][j,:,:] - data["FeuilleUp"][j,:,:] * paramITK["KNUp"] - data["FeuilleUp"][j,:,:] \
            * paramITK["KI"] * data["UBTCulture"][j,:,:] - data["FeuilleUp"][j,:,:] * paramITK["KT"] * data["UBTCulture"][j,:,:],
    )[...,np.newaxis]


    # group 168
    # TigeUp := max(0, (TigeUp -  TigeUp * KNUp - TigeUp * KT * UBTCulture));
    data["TigeUp"][j:,:,:] = np.maximum(
        0,
        data["TigeUp"][j,:,:] - data["TigeUp"][j,:,:] * paramITK["KNUp"] - data["TigeUp"][j,:,:] * paramITK["KT"] * data["UBTCulture"][j,:,:],
    )[...,np.newaxis]
    
    #// D�gradation des feuilles et tiges couch�es (liti�re)
    # group 169
    # LitFeuille :=  max(0, (LitFeuille -  LitFeuille * KNLit - LitFeuille * KI * UBTCulture  - LitFeuille * KT * UBTCulture));
    data["LitFeuille"][j:,:,:] = np.maximum(
        0,
        data["LitFeuille"][j,:,:] - data["LitFeuille"][j,:,:] * paramITK["KNLit"] - data["LitFeuille"][j,:,:] * paramITK["KI"] \
            * data["UBTCulture"][j,:,:] - data["LitFeuille"][j,:,:] * paramITK["KT"] * data["UBTCulture"][j,:,:],
    )[...,np.newaxis]

    # group 170
    # LitTige :=  max(0, (LitTige -  LitTige * KNLit - LitTige * KT * UBTCulture));
    data["LitTige"][j:,:,:] = np.maximum(
        0,
        data["LitTige"][j,:,:] - data["LitTige"][j,:,:] * paramITK["KNLit"] - data["LitTige"][j,:,:] * paramITK["KT"] * data["UBTCulture"][j,:,:],
    )[...,np.newaxis]

    # group 171
    #BiomMc := LitFeuille + LitTige;
    data["biomMc"][j:,:,:] = (data["LitFeuille"][j,:,:] + data["LitTige"][j,:,:])[...,np.newaxis]
     
    # #// transfert dress� � liti�re effet pi�tinement
    # LitFeuille :=  LitFeuille + FeuilleUp * KT * UBTCulture;
    # group 172
    data["LitFeuille"][j:,:,:] = (data["LitFeuille"][j,:,:] + data["FeuilleUp"][j,:,:] * paramITK["KT"] * data["UBTCulture"][j,:,:])[...,np.newaxis]

    # LitTige :=  LitTige + TigeUp * KT * UBTCulture;
    # group 173
    data["LitTige"][j:,:,:] = (data["LitTige"][j,:,:] + data["TigeUp"][j,:,:] * paramITK["KT"] * data["UBTCulture"][j,:,:])[...,np.newaxis]

    # // le 01/03 on consid�re que toutes les pailles et feuilles dressees sont couchees

    #       if (trunc(DayOfTheYear(DateEnCours)) = 61) then
    #   begin
    #     LitFeuille :=  LitFeuille + FeuilleUp;
    #     LitTige :=  LitTige + TigeUp;
    #     FeuilleUp :=  0;
    #     TigeUp :=  0;
    #     BiomMc := LitFeuille + LitTige;
    #  end;

    return data

## MAJBiomMcSV3

In [ ]:
def MAJBiomMcSV3(data):
    """
    groupe 182
 A la Recolte, on calcul la part des biomasses qui restent sur place (Up), non r�colt�es
 et la part qui est mise � terre (Liti�re) sur ce qui est laiss� sur place
 On met a jour aussi la biomasse des liti�res pour les calculs effet mulch sue lr bilan hydrique
    """
#         if (NumPhase =7) then
#     begin
        # groupe 175
#       FeuilleUp := FeuilleUp +  BiomasseFeuilles * (1-TxRecolte);
        # groupe 176
#       TigeUp := TigeUp + BiomasseTiges *  (1-TxRecolte);

        # groupe 177
#       LitFeuille := LitFeuille + FeuilleUp * TxaTerre;

        # groupe 178
#       LitTige := LitTige + TigeUp * TxaTerre;

        # groupe 179
#       FeuilleUp := FeuilleUp * (1-TxaTerre);

        # groupe 180
#       TigeUp := TigeUp * (1-TxaTerre);
# //      LitTige := LitTige + BiomMc;
        # groupe 181
#       BiomMC := LitFeuille + LitTige;
#  {     BiomasseFeuilles := 0;
#       BiomasseTiges := 0;
    return data

In [ ]:
data_2

<xarray.Dataset>
Dimensions:        (x: 239, y: 186, time: 180)
Coordinates:
  * x              (x) float64 -5.963 -5.925 -5.888 -5.85 ... 2.887 2.925 2.962
  * y              (y) float64 15.97 15.94 15.9 15.86 ... 9.15 9.113 9.075 9.038
    spatial_ref    int64 0
Dimensions without coordinates: time
Data variables: (12/13)
    rain           (time, y, x) float32 0.0 0.0 1.6 1.7 1.5 ... 8.6 8.1 6.8 6.2
    tpMoy          (time, y, x) float32 nan nan nan nan ... 24.54 24.54 24.23
    ET0            (time, y, x) float32 nan nan nan nan ... 2.47 2.47 2.47 2.35
    rg             (time, y, x) float32 nan nan nan nan ... 20.4 20.4 20.4 19.83
    irrigation     (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    soil_type      (y, x) float64 11.0 11.0 9.0 9.0 11.0 ... 9.0 9.0 9.0 9.0 9.0
    ...             ...
    epaisseurSurf  (y, x) float64 200.0 200.0 200.0 200.0 ... 200.0 200.0 200.0
    stockIniProf   (y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    stockIniSurf   (y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    seuilRuiss     (y, x) float64 20.0 20.0 18.0 18.0 ... 18.0 18.0 18.0 18.0
    pourcRuiss     (y, x) float64 27.0 27.0 32.0 32.0 ... 32.0 32.0 32.0 32.0
    ru             (y, x) float64 130.0 130.0 155.0 155.0 ... 155.0 155.0 155.0

# Run

In [ ]:
try:
    del data
except:
    pass

In [ ]:
data = data_2.copy(deep=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

data = InitPlotMc(data, grid_width, grid_height, paramITK, paramTypeSol, duration)
data = InitiationCulture(data, grid_width, grid_height, duration, paramVariete)
data = InitSup(data, grid_width, grid_height, duration, paramTypeSol, paramITK)
data = EvalPar(data)


for j in tqdm(range(duration)):

    if date_start + datetime.timedelta(days=j) >= paramITK["DateSemis"]:

        data = EvalPhenoSarrahV3(j, data, paramITK, paramVariete) # ***phenologie*** et exmodules ### trad OK
        data = EvalDegresJourSarrahV3(j, data, paramVariete) # ***phenologie*** et exmodules ### trad OK

    # print(date_start + datetime.timedelta(days=j))
    #bilan hydrique
    data = EvalIrrigPhase(j, data, paramITK)
    data = PluieIrrig(j, data)
    data = RempliMc(j, data, paramITK)
    data = EvalRunOff(j, data, paramTypeSol) 
    data = EvolRurCstr2(j, data, paramITK) 
    data = rempliRes(j, data) 
    data = EvalFESW(j, data) # bhytypeFAO, ***bileau***; exmodules 1 & 2 ###trad O
    data = EvalKceMc(j, data, paramITK) # ***bileau***, exmodules 2 ###trad OK
    data = DemandeSol(j, data) # ***bileau***, exmodules 1 & 2 ### trad OK
    data = EvapMc(j, data, paramITK) # ***bileau***, exmodules 2 ### trad OK
    data = EvapRuSurf(j, data) # ***bileau***, exmodules 1 & 2 ###trad OK
    data = EvalFTSW(j, data) # ***bileau***, exmodules 1 & 2, risocas, riz ###trad OK
    data = EvolKcpKcIni(j, data, paramVariete) # biomasse, exmodules 1 & 2 # comparaison  code nécessaire pour choix ###trad OK
    data = DemandePlante(j, data) # bhytypeFAO, ***bileau***; exmodules 1 & 2 ###trad OK
    data = EvalKcTot(j, data) # ***bileau***, exmodules 1 & 2 ## trad O
    data = CstrPFactor(j, data, paramVariete) # bhytypeFAO, ***bileau***; exmodules 1 & 2, risocas #trad OK
    data = EvalTranspi(j, data) # bhytypeFAO, ***bileau***; exmodules 1 & 2 # trad OK
    data = ConsoResSep(j, data) # ***bileau***; exmodules 1 & 2 # trad O
    # phenologie
    data = EvalVitesseRacSarraV3(j, data, paramVariete) # ***phenologie*** , exmodules 1 & 2 # trad OK
    
    # bilan carbone
    data = EvalLtr(j, data, paramVariete) #biomasse, exmodules 1 & 2 # comparaison code nécessaire pour choix #trad OK
    data = EvalConversion(j, data, paramVariete) # mimlbilancarbone copie, ecopalm2_2, exmodules 1 & 2, ***milbilancarbone***, risocas, riz # trad OK
    data = BiomDensOptSarV42(j, data, paramITK, paramVariete) # ***bilancarbonsarra*** # trad OK
    data = EvalAssimSarrahV42(j, data, paramITK, paramVariete) # ***bilancarbonsarra*** # trad OK
    data = EvalRespMaintSarrahV3(j, data, paramVariete) # ***bilancarbonsarra***, exmodules 1 & 2 ### trad OK
    data = EvolBiomTotSarrahV4(j, data, paramVariete, paramITK) # ***bilancarbonsarra*** ### trad OK, vérifier questiond e la densité
    data = EvalRdtPotRespSarV42(j, data, paramVariete) # ***bilancarbonsarra*** ###trad OK
    data = EvolBiomAeroSarrahV3(j, data, paramVariete) # ***bilancarbonsarra***, exmodules 1 & 2 ###trad OK
    data = EvalReallocationSarrahV3(j, data, paramVariete) # ***bilancarbonsarra***, exmodules 1 & 2 ###trad OK
    data = EvalBiomasseRacinaire(j, data) # copie milbilancarbone, exmodules 1 & 2, ***milbilancarbone*** ### trad OK
    data = EvalFeuilleTigeSarrahV4(j, data, paramVariete) # ***bilancarbonesarra*** ### trad OK
    data = EvalBiomasseVegetati(j, data) # copie milbilancarbon, exmodules 1 & 2, ***milbilancarbone*** ###trad OK
    data = EvalSlaSarrahV3(j, data, paramVariete) # ***bilancarbonesarrah***, exmodules 1 & 2  ### trad OK
    data = EvolLAIPhases(j, data) # exmodules 1 & 2, ***milbilancarbone*** ###trad OK
    data = EvolDayRdtSarraV3(j, data) # ***bilancarbonesarra***, exmodules 1 & 2 ### trad OK
    #phenologie
    data = PhotoperSarrahV3(j, data, paramVariete) # exmodules 1 et 2, ***phenologie*** ###trad OK
    
    # bilan carbone
    data = MortaliteSarraV3(j, data, paramITK, paramVariete) # ***bilancarbonsarra***, exmodules 1 & 2 ### trad OK
    data = BiomDensiteSarraV42(j, data, paramITK, paramVariete)# ***bilancarbonsarra*** ### trad OK
    data = BiomMcUBTSV3(j, data, paramITK) # ***bilancarbonsarra***, exmodules 2
    data = MAJBiomMcSV3(data) # ***bilancarbonsarra***, exmodules 2


  0%|          | 0/180 [00:03<?, ?it/s]


KeyError: 'surface_tank_capacity'

In [ ]:
data

NameError: name 'data' is not defined

In [ ]:
data["rain"][j,:,:] > data["seuilRuiss"]

NameError: name 'data' is not defined

In [ ]:
data["eauDispo"][j:,:,:] = np.maximum(data["eauDispo"][j,:,:] - data["eauCaptee"][j,:,:], 0)

In [ ]:
data["eauDispo"][j:,:,:]

<xarray.DataArray 'eauDispo' (time: 180, y: 186, x: 239)>
array([[[0.        , 0.        , 1.60000002, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.70000005, ..., 0.        ,
         0.        , 0.        ],
        [1.5       , 1.5       , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 1.60000002, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.70000005, ..., 0.        ,
         0.        , 0.        ],
        [1.5       , 1.5       , 0.        , ..., 0.        ,
         0.        , 0.        ],
...
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 1.60000002, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.70000005, ..., 0.        ,
         0.        , 0.        ],
        [1.5       , 1.5       , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]]])
Coordinates:
  * x            (x) float64 -5.963 -5.925 -5.888 -5.85 ... 2.887 2.925 2.962
  * y            (y) float64 15.97 15.94 15.9 15.86 ... 9.15 9.113 9.075 9.038
    spatial_ref  int64 0
Dimensions without coordinates: time
Attributes:
    units:      mm
    long_name:  available water, sum of rainfall and total irrigation for the...

# post-processing : adding crop cover mask

In [ ]:
import xarray as xr 
land_cover_file_path = "/mnt/g/Mon Drive/CIRAD/ESA WorldCover/ESA_worldcover_2021_v200_WestAfrica_preprocessed_for_SARRA-O.tif"
src = xr.open_rasterio(land_cover_file_path)

area = {
    'burkina': [16, -6, 9, 3], #lat,lon lat,lon
    }
selected_area = "burkina"

src = src.where((src.y < area[selected_area][0])
                            & (src.y > area[selected_area][2])
                            & (src.x > area[selected_area][1])
                            & (src.x < area[selected_area][3])
                        ).dropna(dim='y', how='all').dropna(dim='x', how='all')

plt.imshow(src.to_numpy()[0,:,:])



In [ ]:
cropland = np.where(src==40,1,np.nan)[0,:,:]

In [ ]:
plt.imshow(cropland)

In [ ]:
plot_value_mean = data["rdt"].mean(axis=(0,1))
plt.plot(plot_value_mean, label="rdt")

plot_value_mean = np.nanmean((data["rdt"]*cropland[...,np.newaxis]),axis=(0,1))
plt.plot(plot_value_mean, label="rdt x cropland")
plt.legend()
plt.title("rdt (kg/ha) with and without taking in consideration cropland")
plt.show()